In [28]:
from google.cloud import storage
from firebase_admin import firestore, initialize_app

# Establish a connection to the Google Cloud Storage and Firestore
storage_client = storage.Client()
bucket = storage_client.bucket('website-content12345')
initialize_app()
db = firestore.client()

In [4]:
feed = db.collection('feed').document('content-log')
data = feed.get().to_dict()

# Sort by key (timestamp) desc
data = dict(sorted(data.items(), key=lambda item: item[0], reverse=True))
data

{'2024-07-06 17:14:38': {'location': 'blogs/test-blog.md'}}

In [9]:
for key, value in data.items():
    blob = bucket.blob(value['location'])
    md = blob.download_as_string().decode('utf-8')

# We can capture the section between --- and --- and use it as metadata
metadata = md.split('---')[1]
description = {}
for line in metadata.split('\n'):
    if line:
        split_line = line.split(':')
        key = split_line[0].strip()
        # Remove the quotes
        value = ":".join(split_line[1:]).strip()[1:-1]
        description[key.strip()] = value

description

{'title': 'Test Post',
 'author': 'Ed',
 'date': '2024-07-06',
 'tags': '"coding", "python"',
 'type': 'blog',
 'description': 'This is a blog post about coding in Python.',
 'thumbnail': 'images/awesome-blog-post-thumbnail.jpg',
 'og_title': 'Awesome Blog Post',
 'og_description': 'An amazing blog post about coding in Python.',
 'og_image': 'images/awesome-blog-post-og.jpg'}

In [2]:
# Find md files in test-dir
import os
md_files = []
images = []
for root, dirs, files in os.walk('test-dir'):
    for file in files:
        if file.endswith('.md'):
            md_files.append(os.path.join(root, file))
        elif file.endswith('.png') :
            images.append(os.path.join(root, file))

print(md_files)
print("-"*10)
print(images)
print("-"*10)
print(len(md_files))
print("-"*10)
print(len(images))

['test-dir/2024/January/books.md', 'test-dir/2024/February/hip.md', 'test-dir/2024/May/discord.md', 'test-dir/2024/April/internet.md', 'test-dir/2024/March/horses.md', 'test-dir/2023/December/depressed.md', 'test-dir/2023/December/charlmes.md', 'test-dir/2023/July/burnout.md', 'test-dir/2023/July/onrss.md', 'test-dir/2023/July/dustydrawers.md', 'test-dir/2023/August/stress.md', 'test-dir/2023/August/prank.md', 'test-dir/2023/February/stumbleupon.md', 'test-dir/2023/June/alcoholism.md', 'test-dir/2023/June/oldwebsites.md', 'test-dir/2023/April/bananas.md']
----------
['test-dir/blog/2024/phone.png', 'test-dir/blog/2024/horse.png', 'test-dir/blog/2024/skateboard.png', 'test-dir/blog/2024/discord.png', 'test-dir/blog/2023/rss.png', 'test-dir/blog/2023/stumbleUpon.png', 'test-dir/blog/2023/depression.png', 'test-dir/blog/2023/duck.png', 'test-dir/blog/2023/banana.png', 'test-dir/blog/2023/outside.png', 'test-dir/blog/2023/computer.png', 'test-dir/blog/2023/stress.png', 'test-dir/blog/2023/

In [4]:
# Now we need to add metadata to the md files and upload them to the bucket
# The thumbnails can be compressed jpgs of the images
# Oh we also need to link the images in the md files to the image files
related_images = {}
# Go through our markdown files - md file is the key and any images in the doc are the values
for md_file in md_files:
    with open(md_file, 'r') as f:
        md = f.read()
    images = []
    for line in md.split('\n'):
        if '![' in line:
            images.append(line.split('(')[1].split(')')[0])
    related_images[md_file] = {
        'images': images
    }

print(related_images)

{'test-dir/2024/January/books.md': {'images': []}, 'test-dir/2024/February/hip.md': {'images': ['/images/blog/2024/skateboard.png']}, 'test-dir/2024/May/discord.md': {'images': ['/images/blog/2024/discord.png']}, 'test-dir/2024/April/internet.md': {'images': ['/images/blog/2024/phone.png']}, 'test-dir/2024/March/horses.md': {'images': ['/images/blog/2024/horse.png']}, 'test-dir/2023/December/depressed.md': {'images': ['/images/blog/2023/depression.png']}, 'test-dir/2023/December/charlmes.md': {'images': ['/images/blog/2023/duck.png']}, 'test-dir/2023/July/burnout.md': {'images': ['/images/blog/2023/burnout.png']}, 'test-dir/2023/July/onrss.md': {'images': ['/images/blog/2023/rss.png']}, 'test-dir/2023/July/dustydrawers.md': {'images': ['/images/blog/2023/hobbies.png', '/images/blog/2023/outside.png']}, 'test-dir/2023/August/stress.md': {'images': ['/images/blog/2023/stress.png']}, 'test-dir/2023/August/prank.md': {'images': ['/images/blog/2023/computer.png']}, 'test-dir/2023/February/s

In [7]:
from bs4 import BeautifulSoup
metadata = {}
# Cool now we can get the date these were published and the titles
for md_file in md_files:
    with open(md_file, 'r') as f:
        md = f.read()
    # First line is date, rm the # and strip
    date = md.split('\n')[0].strip('#').strip().replace("/", "-")
    # Second line is title, rm the # and strip
    title = md.split('\n')[1].strip('#').strip()

    # We can open the corresponding html file to get the og tags
    html_file = md_file.replace('.md', '.html')
    with open(html_file, 'r') as f:
        html = f.read()

    # Use bs
    soup = BeautifulSoup(html, 'html.parser')
    # Get og tags
    og_tags = {}
    for tag in soup.find_all('meta'):
        if tag.get('property') and tag.get('content'):
            og_tags[tag.get('property').replace(":", "_")] = tag.get('content')

    # Author is always Ed
    author = 'Ed'

    # We need some tags
    # We can set some default tags since this is my peronsal blog about silly things
    tags = ['Silly', 'Personal', 'Lifestyle']

    # Type is blog
    _type = 'blog'

    # Now we just need a thumbnail which we'll copy the first image name, add _thumbnail, compress to 64x64 and make a .jpg
    try:
        thumbnail = related_images[md_file]['images'][0].replace('.png', '_thumbnail.jpg')
    except:
        thumbnail = None

    # Now set all this data
    data = {
        'date': date,
        'title': title,
        'author': author,
        'tags': tags,
        'type': _type,
        'thumbnail': thumbnail,
        'og_tags': og_tags
    }

    metadata[md_file] = data
    metadata[md_file]['images'] = related_images[md_file]['images']

metadata

{'test-dir/2024/January/books.md': {'date': '04-01-2024',
  'title': 'Best Books of 2023',
  'author': 'Ed',
  'tags': ['Silly', 'Personal', 'Lifestyle'],
  'type': 'blog',
  'thumbnail': None,
  'og_tags': {'og_title': 'Best Books of 2023',
   'og_description': 'I started 2023 off strong with my reading, absolutely ploughing through books at a rate of one every other day, but this significantly slowed down in September ...',
   'og_type': 'article'},
  'images': []},
 'test-dir/2024/February/hip.md': {'date': '01-02-2024',
  'title': 'Why Do I Keep Injuring My Left Hip Specifically?',
  'author': 'Ed',
  'tags': ['Silly', 'Personal', 'Lifestyle'],
  'type': 'blog',
  'thumbnail': '/images/blog/2024/skateboard_thumbnail.jpg',
  'og_tags': {'og_title': 'Why Do I Keep Injuring My Left Hip Specifically?',
   'og_image': '/images/blog/2024/skateboard.png',
   'og_description': "When I was 30 years old I had a great idea. In spite of having never rollerskated in my life (apart from apparent

In [14]:
# Now we can insert this as metadata into our blog markdown and move them up to the root directory and also strip all the path from the image paths except filename
# Also images doesn't need to be included in the metadata it's just so we can track what goes in the bucket
# All our md files need the image paths setting to /assets/images/imagename.png
# With that in mind let's go ahead and do it
for md_file in md_files:
    with open(md_file, 'r') as f:
        md = f.read()

    # Remove the first two lines as these are title/date which we'll render from the metadata
    md = "\n".join(md.split('\n')[2:])

    # Now strip any excess lines
    md = md.strip()

    # Replace the image paths
    for image in metadata[md_file]['images']:
        md = md.replace(image, f"/assets/images/{image.split('/')[-1]}")

    # Get the metadata
    data = metadata[md_file]

    # Now we need to update the md file with the metadata
    new_md = f"""---
date: {data['date']}
title: {data['title']}
author: {data['author']}
tags: {data['tags']}
type: {data['type']}
thumbnail: /assets/images/{data['thumbnail'].split('/')[-1] if data['thumbnail'] else ''}
og_title: {data['og_tags'].get('og_title', '')}
og_description: {data['og_tags'].get('og_description', '')}
og_image: {data['og_tags'].get('og_image', '')}
og_type: {data['og_tags'].get('og_type', '')}
---
{md}
"""

    if not os.path.exists('test-dir/blogs'):
        os.makedirs('test-dir/blogs')

    # Write this new md to test-dir/blogs
    with open(f"test-dir/blogs/{md_file.split('/')[-1]}", 'w') as f:
        f.write(new_md)

# Now let's copy all our images to test-dir/images
import shutil
if not os.path.exists('test-dir/images'):
    os.makedirs('test-dir/images')

for image in images:
    shutil.copy(os.path.join('test-dir', *image.split('/')[2:]), f"test-dir/images/{image.split('/')[-1]}")

In [16]:
images = []
for root, dirs, files in os.walk('test-dir'):
    for file in files:
        if file.endswith('.md'):
            md_files.append(os.path.join(root, file))
        elif file.endswith('.png') :
            images.append(os.path.join(root, file))

In [19]:

for image in images:
    try:
        shutil.copy(os.path.join('test-dir', *image.split('/')[1:]), f"test-dir/images/{image.split('/')[-1]}")
    except Exception as e:
        print(e)


'test-dir/images/banana.png' and 'test-dir/images/banana.png' are the same file


In [20]:
# Date format is incorrect it goes dd-mm-yyyy instead of yyyy-mm-dd
# Let's fix that
for root, dirs, files in os.walk('test-dir/blogs'):
    for file in files:
        with open(os.path.join(root, file), 'r') as f:
            md = f.read()
        date = md.split('date: ')[1].split('\n')[0]
        date = date.split('-')
        date = f"{date[2]}-{date[1]}-{date[0]}"
        md = md.replace(md.split('date: ')[1].split('\n')[0], date)
        with open(os.path.join(root, file), 'w') as f:
            f.write(md)


In [21]:
# Oh let's use PIL to make our thumbnail images
images = os.listdir('test-dir/images')
images

['phone.png',
 'horse.png',
 'rss.png',
 'stumbleUpon.png',
 'depression.png',
 'duck.png',
 'banana.png',
 'outside.png',
 'computer.png',
 'stress.png',
 'burnout.png',
 'hu.png',
 'skateboard.png',
 'stairs.png',
 'hobbies.png',
 'discord.png']

In [27]:
from PIL import Image
for image in images:
    img = Image.open(f"test-dir/images/{image}")
    img.thumbnail((256, 256))
    img = img.convert("RGB")
    img.save(f"test-dir/images/{image.replace('.png', '_thumbnail.jpg')}")

In [39]:
import re

pattern = r'(og_image: )(/images/blog/\d{4}/)'

# Our md files have incorrect image paths for og_image
for md_file in os.listdir(os.path.join('test-dir', 'blogs')):
    with open(os.path.join('test-dir', 'blogs', md_file), 'r') as f:
        md = f.read()

    # Regex replace the capture group with /assets/images/
    md = re.sub(pattern, r'\1/assets/images/', md)

    # Write it back
    with open(os.path.join('test-dir', 'blogs', md_file), 'w') as f:
        f.write(md)


In [42]:
# Cool now we need to write our firestore log
feed = db.collection('feed').document('content-log')
data = feed.get().to_dict()
data

{'2024-07-06 17:14:38': {'location': 'blogs/test-blog.md'}}

In [44]:
from datetime import datetime as dt
from datetime import timedelta

data = {}
# So we will overwrite this with all our blogs - we do actually have a last edit date in the original file metadata we can use
# But if it is prior to the actual date we will use the actual date at midnight
# So let's go ahead and do that
# First go through and get datetime objects for all the blogs
for year in ['2023', '2024']:
    for root, dirs, files in os.walk(os.path.join('test-dir', year)):
        for file in files:
            if file.endswith('.md'):
                with open(os.path.join(root, file), 'r') as f:
                    md = f.read()
                actual_date = md.split('\n')[0].strip('#').strip().replace("/", "-")
                # Parse the actual date
                actual_date = dt.strptime(actual_date, "%d-%m-%Y")
                # Set time to midnight
                actual_date = actual_date.replace(hour=0, minute=0, second=0, microsecond=0)

                # Now get the file edit datetime
                edited_time = dt.fromtimestamp(os.path.getmtime(os.path.join(root, file)))

                # Make sure that edited_time isn't more than 1 day ahead of actual_date
                if edited_time <= actual_date + timedelta(days=1):
                    actual_date = edited_time

                data[actual_date.strftime("%Y-%m-%d %H:%M:%S")] = {
                    'location': os.path.join('blogs', file),
                }

print(data)

{'2023-12-17 16:08:05': {'location': 'blogs/depressed.md'}, '2023-12-23 11:03:45': {'location': 'blogs/charlmes.md'}, '2023-07-23 10:22:07': {'location': 'blogs/burnout.md'}, '2023-07-18 08:52:17': {'location': 'blogs/onrss.md'}, '2023-07-29 09:05:47': {'location': 'blogs/dustydrawers.md'}, '2023-08-08 12:02:16': {'location': 'blogs/stress.md'}, '2023-08-22 15:15:32': {'location': 'blogs/prank.md'}, '2023-02-22 00:00:00': {'location': 'blogs/stumbleupon.md'}, '2023-06-05 00:00:00': {'location': 'blogs/alcoholism.md'}, '2023-06-20 14:28:27': {'location': 'blogs/oldwebsites.md'}, '2023-04-17 00:00:00': {'location': 'blogs/bananas.md'}, '2024-01-04 11:35:56': {'location': 'blogs/books.md'}, '2024-02-01 17:08:24': {'location': 'blogs/hip.md'}, '2024-05-06 09:38:07': {'location': 'blogs/discord.md'}, '2024-04-23 08:17:04': {'location': 'blogs/internet.md'}, '2024-03-14 00:00:00': {'location': 'blogs/horses.md'}}


In [45]:
# Banging job now let's just order it such that the most recent is first
data = dict(sorted(data.items(), key=lambda item: item[0], reverse=True))
data

{'2024-05-06 09:38:07': {'location': 'blogs/discord.md'},
 '2024-04-23 08:17:04': {'location': 'blogs/internet.md'},
 '2024-03-14 00:00:00': {'location': 'blogs/horses.md'},
 '2024-02-01 17:08:24': {'location': 'blogs/hip.md'},
 '2024-01-04 11:35:56': {'location': 'blogs/books.md'},
 '2023-12-23 11:03:45': {'location': 'blogs/charlmes.md'},
 '2023-12-17 16:08:05': {'location': 'blogs/depressed.md'},
 '2023-08-22 15:15:32': {'location': 'blogs/prank.md'},
 '2023-08-08 12:02:16': {'location': 'blogs/stress.md'},
 '2023-07-29 09:05:47': {'location': 'blogs/dustydrawers.md'},
 '2023-07-23 10:22:07': {'location': 'blogs/burnout.md'},
 '2023-07-18 08:52:17': {'location': 'blogs/onrss.md'},
 '2023-06-20 14:28:27': {'location': 'blogs/oldwebsites.md'},
 '2023-06-05 00:00:00': {'location': 'blogs/alcoholism.md'},
 '2023-04-17 00:00:00': {'location': 'blogs/bananas.md'},
 '2023-02-22 00:00:00': {'location': 'blogs/stumbleupon.md'}}

In [46]:
# Slam it into firestore
feed.set(data)

update_time {
  seconds: 1720885488
  nanos: 98901000
}

In [51]:
# Go through and copy the og_description to the description field
for md_file in os.listdir(os.path.join('test-dir', 'blogs')):
    with open(os.path.join('test-dir', 'blogs', md_file), 'r') as f:
        md = f.read()

    # Get the og_description
    og_description = md.split('og_description: ')[1].split('\n')[0]

    # Add a description field under title
    title = md.split('title: ')[1].split('\n')[0]
    title_with_description = f"{title}\ndescription: {og_description}"
    md = md.replace(title, title_with_description)

    # Write it back
    with open(os.path.join('test-dir', 'blogs', md_file), 'w') as f:
        f.write(md)

In [53]:
# Rename our music files to add underscores in place of spaces
for root, dirs, files in os.walk('test-dir/music'):
    for file in files:
        if ' ' in file:
            os.rename(os.path.join(root, file), os.path.join(root, file.replace(' ', '_')))
        if '[' in file or ']' in file:
            os.rename(os.path.join(root, file), os.path.join(root, file.replace('[', '').replace(']', '')))

In [54]:
# Let's write a markdown file for our Planet Ed Album
metadata = {
    'date': '2005-01-01',
    'title': 'Planet Ed',
    'description': 'My first album, Planet Ed, was released in 2005. Nobody really cared. I was 12 going on 13.',
    'author': 'Ed',
    'tags': ['Music', 'Planet Ed'],
    'type': 'music',
    'thumbnail': '/assets/images/planet_ed.jpg',
    'og_title': 'Planet Ed',
    'og_description': 'My first album, Planet Ed, was released in 2005. Nobody really cared. I was 12 going on 13.',
    'og_image': '/assets/images/planet_ed.jpg',
    'og_type': 'music'
}

metadata_md = f"""---
date: {metadata['date']}
title: {metadata['title']}
description: {metadata['description']}
author: {metadata['author']}
tags: {metadata['tags']}
type: {metadata['type']}
thumbnail: {metadata['thumbnail']}
og_title: {metadata['og_title']}
og_description: {metadata['og_description']}
og_image: {metadata['og_image']}
og_type: {metadata['og_type']}
---
"""

# Now the actual content
content = """
---
When I was a young 12 year old, I was playing around with Gamemaker.

I took copyright law very seriously, and decided I must make my own music for my games otherwise I would be a criminal.

Thus started my music career, under the moniker Planet Ed (at some point stylised instead as Planet 'ed, a contraction of Planet Head). I didn't know anything about music theory. I had a light background in jazz piano, but I generally couldn't be bothered to learn anything properly.

Destroyed World was the first song I made, I was playing Jak and Daxter at the time so I was inspired by the music in that game. It is unexpectedly a very good piece of music.

The rest of the music may seem weird and disjointed, but I was 12 and to reiterate, I didn't know anything about music theory.

Armed with a copy of Sibelius 3 and an EMU Proteus 2000, I set out to make my first album. I was very proud of it at the time. Now I just look back at it with nostalgia.

I hope you enjoy it.

PS. The recording is terrible because I didn't know how to record properly. I didn't know what a DAW was.
---
title: Blips
file: /assets/music/01_Blips.mp3
title: Heaven
file: /assets/music/02_Heaven.mp3
title: Drum Pie and Peas
file: /assets/music/03_Drum_Pie_and_Peas.mp3
title: Folk Thing
file: /assets/music/04_Folk_Thing.mp3
title: Power Plant [Power Failure Remix]
file: /assets/music/05_Power_Plant_Power_Failure_Remix.mp3
title: Heavy Machinery
file: /assets/music/06_Heavy_Machinery.mp3
title: Aliens
file: /assets/music/07_Aliens.mp3
title: Epic Song
file: /assets/music/08_Epic_Song.mp3
title: Destroyed World
file: /assets/music/09_Destroyed_World.mp3
title: The Banjo Experience
file: /assets/music/10_The_Banjo_Experience.mp3
title: Taking a Ride
file: /assets/music/11_Taking_a_Ride.mp3
title: Power Plant
file: /assets/music/12_Power_Plant.mp3
"""

with open('test-dir/planet_ed.md', 'w') as f:
    f.write(metadata_md + content)

In [58]:
# Alright we gonna add collection metadata to our markdown files
for f in os.listdir(os.path.join('test-dir', 'blogs')):
    with open(os.path.join('test-dir', 'blogs', f), 'r') as file:
        md = file.read()

    # Add collection metadata
    collection_metadata = "collection: Ed's Blog"
    metadata = md.split('---')[1]
    md = md.replace(metadata, f"{metadata}{collection_metadata}\n")
    # Write it back
    with open(os.path.join('test-dir', 'blogs', f), 'w') as file:
        file.write(md)

# Now same in the music directory
with open('test-dir/planet_ed.md', 'r') as file:
    md = file.read()
metadata = md.split('---')[1]
collection_metadata = "collection: Ed's Music"
# Add collection metadata
md = md.replace(metadata, f"{metadata}{collection_metadata}\n")
# Write it back
with open('test-dir/planet_ed.md', 'w') as file:
    file.write(md)

In [60]:
# Now let's get our feed data and create a list of all our blog posts in reverse chronological order
# We can use this to write to our new firestore collection
feed = db.collection('feed').document('content-log').get().to_dict()
feed

{'2023-02-22 00:00:00': {'location': 'blogs/stumbleupon.md'},
 '2024-03-14 00:00:00': {'location': 'blogs/horses.md'},
 '2023-12-23 11:03:45': {'location': 'blogs/charlmes.md'},
 '2024-04-23 08:17:04': {'location': 'blogs/internet.md'},
 '2023-06-20 14:28:27': {'location': 'blogs/oldwebsites.md'},
 '2024-02-01 17:08:24': {'location': 'blogs/hip.md'},
 '2023-12-17 16:08:05': {'location': 'blogs/depressed.md'},
 '2023-04-17 00:00:00': {'location': 'blogs/bananas.md'},
 '2024-01-04 11:35:56': {'location': 'blogs/books.md'},
 '2023-07-29 09:05:47': {'location': 'blogs/dustydrawers.md'},
 '2023-07-18 08:52:17': {'location': 'blogs/onrss.md'},
 '2023-08-22 15:15:32': {'location': 'blogs/prank.md'},
 '2023-08-08 12:02:16': {'location': 'blogs/stress.md'},
 '2023-06-05 00:00:00': {'location': 'blogs/alcoholism.md'},
 '2023-07-23 10:22:07': {'location': 'blogs/burnout.md'},
 '2024-05-06 09:38:07': {'location': 'blogs/discord.md'}}

In [63]:
new_doc = db.collection('collections').document('eds_blog')
# Let's write a list from our feed data
data = [
    v['location'].split('/')[-1] for v in feed.values()
]
data

['stumbleupon.md',
 'horses.md',
 'charlmes.md',
 'internet.md',
 'oldwebsites.md',
 'hip.md',
 'depressed.md',
 'bananas.md',
 'books.md',
 'dustydrawers.md',
 'onrss.md',
 'prank.md',
 'stress.md',
 'alcoholism.md',
 'burnout.md',
 'discord.md']

In [64]:
# Cool now we can write this to our new collection
new_doc.set({'content': data})

update_time {
  seconds: 1720953587
  nanos: 833201000
}

In [69]:
# We should also add the music album to the music collection
new_doc = db.collection('collections').document('eds_music')
new_doc.set({'content': ['planet_ed.md']})


update_time {
  seconds: 1720957229
  nanos: 499697000
}

In [73]:
# Let's add our music to the feed
feed = db.collection('feed').document('content-log')
data = feed.get().to_dict()
data['2005-01-01 00:00:00'] = {
    'location': 'music/planet_ed.md'
}

data

{'2023-02-22 00:00:00': {'location': 'blogs/stumbleupon.md'},
 '2024-03-14 00:00:00': {'location': 'blogs/horses.md'},
 '2023-12-23 11:03:45': {'location': 'blogs/charlmes.md'},
 '2024-04-23 08:17:04': {'location': 'blogs/internet.md'},
 '2023-06-20 14:28:27': {'location': 'blogs/oldwebsites.md'},
 '2024-02-01 17:08:24': {'location': 'blogs/hip.md'},
 '2023-12-17 16:08:05': {'location': 'blogs/depressed.md'},
 '2023-04-17 00:00:00': {'location': 'blogs/bananas.md'},
 '2024-01-04 11:35:56': {'location': 'blogs/books.md'},
 '2023-07-29 09:05:47': {'location': 'blogs/dustydrawers.md'},
 '2023-07-18 08:52:17': {'location': 'blogs/onrss.md'},
 '2023-08-22 15:15:32': {'location': 'blogs/prank.md'},
 '2023-08-08 12:02:16': {'location': 'blogs/stress.md'},
 '2023-06-05 00:00:00': {'location': 'blogs/alcoholism.md'},
 '2023-07-23 10:22:07': {'location': 'blogs/burnout.md'},
 '2024-05-06 09:38:07': {'location': 'blogs/discord.md'},
 '2005-01-01 00:00:00': {'location': 'music/planet_ed.md'}}

In [74]:
# Put it back
feed.set(data)

update_time {
  seconds: 1720957492
  nanos: 514354000
}

In [82]:
from random import randint
# Let's try add some comics (Hewligg Urubokkle the old shit comics I made when I was 12)
# Alright we need to go through images and create markdown documents for every page, they don't have names so just call them 'Hewligg Urobokkle 1' etc
for im in os.listdir(os.path.join('STAGING', 'images')):
    filename = im.split(".")[0]

    new_filename = 'hewligg_urobokkle_' + im
    # Rename the image using shutil
    shutil.move(os.path.join('STAGING','images', im), os.path.join('STAGING', 'images', 'hewligg_urobokkle_' + im))
    im = new_filename
    # Make a thumbnail by sampling some of the comic
    img = Image.open(os.path.join('STAGING', 'images', im))
    img = img.convert('RGB')
    # Subsample 256x256 of the page
    width, height = img.size
    max_width = width - 256
    max_height = height - 256
    x0 = randint(0, max_width)
    y0 = randint(0, max_height)
    x1 = x0 + 256
    y1 = y0 + 256
    bounds = (x0, y0, x1, y1)
    cropped_img = img.crop(bounds)
    cropped_img.thumbnail((256,256))

    # Save into the staging area
    cropped_img.save(os.path.join("STAGING", "images", im.split('.')[0] + '_thumbnail.jpg'))
    if '_' in filename:
        filename = " Part ".join(filename.split("_"))
    md = f"""
---
date: YYYY-MM-DD
title: Hewligg Urobokkle {filename}
description: Description
author: Ed
tags: ['Sprite Comic', 'Archive', 'Hewligg Urobokkle']
type: comic
thumbnail: /assets/images/{im.split('.')[0]}_thumbnail.jpg
og_title: Hewligg Urobokkle {filename}
og_description: Description
og_image: /assets/images/{im.split('.')[0]}_thumbnail.jpg
og_type: article
collection: Hewligg Urobokkle
---
# Hewligg Urobokkle {filename}

![Hewligg Urobokkle {filename}](/assets/images/{im})
"""

    with open(os.path.join('STAGING', 'comics', im.split('.')[0] + '.md'), 'w') as f:
        f.write(md)

In [85]:
# Okay so we can now write the metadata for the Hewligg Urobokkle collection
collection = db.collection('collections').document('hewligg_urobokkle')
data = [
    f for f in os.listdir(os.path.join('STAGING', 'comics'))
]
# Sort it based on the number in the title
data = sorted(data, key=lambda x: float('_'.join(x.split('_')[2:]).split('.')[0].replace('_','.')))
data

['hewligg_urobokkle_1.md',
 'hewligg_urobokkle_2.md',
 'hewligg_urobokkle_3.md',
 'hewligg_urobokkle_4.md',
 'hewligg_urobokkle_5.md',
 'hewligg_urobokkle_6.md',
 'hewligg_urobokkle_7.md',
 'hewligg_urobokkle_8.md',
 'hewligg_urobokkle_9.md',
 'hewligg_urobokkle_10.md',
 'hewligg_urobokkle_11.md',
 'hewligg_urobokkle_12.md',
 'hewligg_urobokkle_13.md',
 'hewligg_urobokkle_14.md',
 'hewligg_urobokkle_15.md',
 'hewligg_urobokkle_16.md',
 'hewligg_urobokkle_17.md',
 'hewligg_urobokkle_18.md',
 'hewligg_urobokkle_19.md',
 'hewligg_urobokkle_20_1.md',
 'hewligg_urobokkle_20_2.md',
 'hewligg_urobokkle_20_3.md',
 'hewligg_urobokkle_21_1.md',
 'hewligg_urobokkle_21_2.md',
 'hewligg_urobokkle_21_3.md',
 'hewligg_urobokkle_22_1.md',
 'hewligg_urobokkle_22_2.md',
 'hewligg_urobokkle_23.md',
 'hewligg_urobokkle_24.md',
 'hewligg_urobokkle_25.md',
 'hewligg_urobokkle_26.md',
 'hewligg_urobokkle_27.md',
 'hewligg_urobokkle_28.md',
 'hewligg_urobokkle_29.md',
 'hewligg_urobokkle_30.md',
 'hewligg_uro

In [86]:
# Write it back
collection.set({'content': data})

update_time {
  seconds: 1720964939
  nanos: 861734000
}

In [87]:
# Great now let's add our comments to the feed based on the date of the comic
# But this has slightly more nuance we need a datetime object for the date of the comic
# Some share a date so we need to add a second to the time
feed = db.collection('feed').document('content-log')
feed_data = feed.get().to_dict()
feed_data

{'2023-02-22 00:00:00': {'location': 'blogs/stumbleupon.md'},
 '2024-03-14 00:00:00': {'location': 'blogs/horses.md'},
 '2024-04-23 08:17:04': {'location': 'blogs/internet.md'},
 '2023-12-23 11:03:45': {'location': 'blogs/charlmes.md'},
 '2023-06-20 14:28:27': {'location': 'blogs/oldwebsites.md'},
 '2024-02-01 17:08:24': {'location': 'blogs/hip.md'},
 '2023-12-17 16:08:05': {'location': 'blogs/depressed.md'},
 '2023-04-17 00:00:00': {'location': 'blogs/bananas.md'},
 '2024-01-04 11:35:56': {'location': 'blogs/books.md'},
 '2023-07-29 09:05:47': {'location': 'blogs/dustydrawers.md'},
 '2023-07-18 08:52:17': {'location': 'blogs/onrss.md'},
 '2023-08-22 15:15:32': {'location': 'blogs/prank.md'},
 '2023-08-08 12:02:16': {'location': 'blogs/stress.md'},
 '2023-06-05 00:00:00': {'location': 'blogs/alcoholism.md'},
 '2005-01-01 00:00:00': {'location': 'music/planet_ed.md'},
 '2024-05-06 09:38:07': {'location': 'blogs/discord.md'},
 '2023-07-23 10:22:07': {'location': 'blogs/burnout.md'}}

In [91]:
# Now let's create a dict of the comic data
new_feed_data = {}
for comic in data:
    with open(os.path.join('STAGING', 'comics', comic), 'r') as f:
        md = f.read()

    # Get the date
    date = md.split('date: ')[1].split('\n')[0]
    date = dt.strptime(date, "%Y-%m-%d")
    while date.strftime("%Y-%m-%d %H:%M:%S") in new_feed_data:
        date = date + timedelta(seconds=1)

    new_feed_data[date.strftime("%Y-%m-%d %H:%M:%S")] = {
        'location': os.path.join('comics', comic)
    }

assert len(new_feed_data.keys()) == len(data)

In [93]:
# Now extend feed_data
feed_data.update(new_feed_data)
feed_data

{'2023-02-22 00:00:00': {'location': 'blogs/stumbleupon.md'},
 '2024-03-14 00:00:00': {'location': 'blogs/horses.md'},
 '2024-04-23 08:17:04': {'location': 'blogs/internet.md'},
 '2023-12-23 11:03:45': {'location': 'blogs/charlmes.md'},
 '2023-06-20 14:28:27': {'location': 'blogs/oldwebsites.md'},
 '2024-02-01 17:08:24': {'location': 'blogs/hip.md'},
 '2023-12-17 16:08:05': {'location': 'blogs/depressed.md'},
 '2023-04-17 00:00:00': {'location': 'blogs/bananas.md'},
 '2024-01-04 11:35:56': {'location': 'blogs/books.md'},
 '2023-07-29 09:05:47': {'location': 'blogs/dustydrawers.md'},
 '2023-07-18 08:52:17': {'location': 'blogs/onrss.md'},
 '2023-08-22 15:15:32': {'location': 'blogs/prank.md'},
 '2023-08-08 12:02:16': {'location': 'blogs/stress.md'},
 '2023-06-05 00:00:00': {'location': 'blogs/alcoholism.md'},
 '2005-01-01 00:00:00': {'location': 'music/planet_ed.md'},
 '2024-05-06 09:38:07': {'location': 'blogs/discord.md'},
 '2023-07-23 10:22:07': {'location': 'blogs/burnout.md'},
 '20

In [94]:
# Cool let's put it back
feed.set(feed_data)

update_time {
  seconds: 1720965142
  nanos: 440245000
}

In [97]:
# We need to update the feed again because hewligg_urobokkle_21_3.md said 2024 instead of 2004
feed = db.collection('feed').document('content-log')
feed_data = feed.get().to_dict()
feed_data['2024-08-23 00:00:00']

{'location': 'comics/hewligg_urobokkle_21_3.md'}

In [98]:
# Let's pop that key and reinsert it
old_data = feed_data.pop('2024-08-23 00:00:00')
feed_data['2004-08-23 00:00:00'] = old_data
feed_data


{'2004-08-16 00:00:07': {'location': 'comics/hewligg_urobokkle_8.md'},
 '2024-04-23 08:17:04': {'location': 'blogs/internet.md'},
 '2004-08-16 00:00:01': {'location': 'comics/hewligg_urobokkle_2.md'},
 '2024-02-01 17:08:24': {'location': 'blogs/hip.md'},
 '2004-10-15 00:00:00': {'location': 'comics/hewligg_urobokkle_38.md'},
 '2023-07-18 08:52:17': {'location': 'blogs/onrss.md'},
 '2024-01-04 11:35:56': {'location': 'blogs/books.md'},
 '2004-10-21 00:00:00': {'location': 'comics/hewligg_urobokkle_44.md'},
 '2004-08-17 00:00:03': {'location': 'comics/hewligg_urobokkle_13.md'},
 '2004-10-06 00:00:00': {'location': 'comics/hewligg_urobokkle_28.md'},
 '2004-10-27 00:00:00': {'location': 'comics/hewligg_urobokkle_50.md'},
 '2004-10-10 00:00:00': {'location': 'comics/hewligg_urobokkle_32.md'},
 '2004-08-18 00:00:01': {'location': 'comics/hewligg_urobokkle_17.md'},
 '2004-08-19 00:00:00': {'location': 'comics/hewligg_urobokkle_20_3.md'},
 '2023-02-22 00:00:00': {'location': 'blogs/stumbleupon

In [99]:
assert len(feed.get().to_dict()) == len(feed_data)

In [101]:
# Cool let's put it back
feed.set(feed_data)

update_time {
  seconds: 1720965936
  nanos: 530778000
}

In [106]:
# The list of blogs is out of order so let's fix that
blogs_list = []
# Sort feed_data by date - oldest first
feed_data = dict(sorted(feed_data.items(), key=lambda item: item[0]))
for k, v in feed_data.items():
    location = v['location']
    if 'blogs' in location:
        print(k)
        blogs_list.append(location.split('/')[-1])

blogs_list

2023-02-22 00:00:00
2023-04-17 00:00:00
2023-06-05 00:00:00
2023-06-20 14:28:27
2023-07-18 08:52:17
2023-07-23 10:22:07
2023-07-29 09:05:47
2023-08-08 12:02:16
2023-08-22 15:15:32
2023-12-17 16:08:05
2023-12-23 11:03:45
2024-01-04 11:35:56
2024-02-01 17:08:24
2024-03-14 00:00:00
2024-04-23 08:17:04
2024-05-06 09:38:07


['stumbleupon.md',
 'bananas.md',
 'alcoholism.md',
 'oldwebsites.md',
 'onrss.md',
 'burnout.md',
 'dustydrawers.md',
 'stress.md',
 'prank.md',
 'depressed.md',
 'charlmes.md',
 'books.md',
 'hip.md',
 'horses.md',
 'internet.md',
 'discord.md']

In [107]:
# Okay now blogs list looks right let's put it back
new_doc = db.collection('collections').document('eds_blog')
new_doc.set({'content': blogs_list})

update_time {
  seconds: 1720968778
  nanos: 830024000
}

In [116]:
# Okay so currently our comics have a headig but the jinja2 layout deals with the heading so let's strip that out our markdown files
for md_file in os.listdir(os.path.join('CONTENT', 'comics')):
    with open(os.path.join('CONTENT', 'comics', md_file), 'r') as f:
        md = f.read()

    # Split on ---
    nothing, metadata, content = md.split('---')
    reconstructed_content = ''
    for line in content.split('\n'):
        if not line:
            continue
        elif line.startswith('#'):
            continue
        reconstructed_content += line + '\n'

    # Strip newlines
    reconstructed_content = '\n' + reconstructed_content.strip()

    # Join it back together
    md = '---'.join([nothing, metadata, reconstructed_content])

    # Write it back
    with open(os.path.join('CONTENT', 'comics', md_file), 'w') as f:
        f.write(md)


In [117]:
# Now deal with pixelated peculirarities
import json
with open(os.path.join('STAGING', 'comic_data.json'), 'r') as f:
    comic_data = json.loads(f.read())
comic_data

[{'name': 'Sonic the Hedgehog in Chilli Dog Zone',
  'description': "Sonic the Hedgehog tells Commander Keen to have a chilli dog but finds out he doesn't know what they are, so they go to Chilli Dog Zone.",
  'date': '2023-07-03',
  'src': 'comic1.png',
  'tooltip': 'Sonk 3'},
 {'name': 'Gotta Grow Pasta',
  'description': "Knuckles the Echidna tries to punch Sonic but it doesn't work because he's too fast, so he grows pasta.",
  'date': '2023-07-04',
  'src': 'comic2.png',
  'tooltip': 'BBC pasta woman'},
 {'name': "Alex Kidd's Whacky Wheelie Adventure",
  'description': "Alex Kidd drives a car and it's whacky and wheelie.",
  'date': '2023-07-04',
  'src': 'comic3.png',
  'tooltip': 'Read Alex Kidd Buys Heroin'},
 {'name': 'Guybrush Threepwood and the Ordinary Rubber Chicken With a Pulley in the Middle of Th eK',
  'description': "Guybrush Threepwood finds a perfectly normal rurbber chicken with a pulley in the middle. OR DOES HE? (He does.) Oh, and there's a monkey. And a pirate. n

In [121]:
# Okay so we gotta loop over this, generate metadata, generate content, add hover_text, and link up the thumbnail and og tag
feed = {}
for comic in comic_data:
    metadata = {}
    metadata['title'] = comic['name']
    metadata['description'] = comic['description']
    metadata['date'] = comic['date']
    metadata['hover_text'] = comic['tooltip']
    metadata['author'] = 'Ed'
    metadata['tags'] = ['Sprite Comic', 'Irony']
    metadata['type'] = 'comic'
    metadata['og_title'] = comic['name']
    metadata['og_description'] = comic['description']
    metadata['og_type'] = 'article'
    metadata['collection'] = 'Pixelated Peculirarities'

    # Now generate the thumbnail and organise the images
    og_name = 'comic_' + comic['src'].split('comic')[1]
    og_img = os.path.join('STAGING', 'og', og_name)
    new_name = 'pp_'+og_name.split('.')[0] + '_og.jpg'
    shutil.copy(og_img, os.path.join('STAGING', 'images', new_name))
    metadata['og_image'] = f"/assets/images/{new_name}"

    # Thumbnail
    img = Image.open(og_img)
    img.thumbnail((256, 256))
    img = img.convert('RGB')

    thumbnail_name = 'pp_'+og_name.split('.')[0] + '_thumbnail.jpg'
    img.save(os.path.join('STAGING', 'images', thumbnail_name))

    metadata['thumbnail'] = f"/assets/images/{thumbnail_name}"

    # Move the src image to the images directory with a new name
    comic_filename = f'pp_{comic["src"]}'
    shutil.copy(os.path.join('STAGING', 'img', comic['src']), os.path.join('STAGING', 'images', comic_filename))

    # Now finally generate the markdown doc
    content = f"""---
date: {metadata['date']}
title: {metadata['title']}
description: {metadata['description']}
author: {metadata['author']}
tags: {metadata['tags']}
type: {metadata['type']}
thumbnail: {metadata['thumbnail']}
og_title: {metadata['og_title']}
og_description: {metadata['og_description']}
og_image: {metadata['og_image']}
og_type: {metadata['og_type']}
collection: {metadata['collection']}
hover_text: {metadata['hover_text']}
---
![{metadata['title']}](/assets/images/{comic_filename})
"""

    with open(os.path.join('STAGING', 'comics', comic_filename.split('.')[0] + '.md'), 'w') as f:
        f.write(content)

    # Now get the datetime of edit from the file src
    edited_time = dt.fromtimestamp(os.path.getmtime(os.path.join('STAGING', 'img', comic['src'])))

    str_time = edited_time.strftime("%Y-%m-%d %H:%M:%S")

    # Add to feed
    feed[str_time] = {
        'location': os.path.join('comics', comic_filename.split('.')[0] + '.md')
    }

In [122]:
# Finally we need a content feed for the Pixelated Peculirarities collection
new_doc = db.collection('collections').document('pixelated_peculirarities')
data = [
    f for f in os.listdir(os.path.join('STAGING', 'comics')) if not 'thumbnail' in f and not 'og' in f
]
data

['pp_comic12.md',
 'pp_comic17.md',
 'pp_comic8.md',
 'pp_comic11.md',
 'pp_comic5.md',
 'pp_comic4.md',
 'pp_comic10.md',
 'pp_comic6.md',
 'pp_comic9.md',
 'pp_comic7.md',
 'pp_comic3.md',
 'pp_comic16.md',
 'pp_comic2.md',
 'pp_comic15.md',
 'pp_comic13.md',
 'pp_comic1.md',
 'pp_comic14.md']

In [124]:
# This data needs to be sorted by date
data = sorted(data, key=lambda x: int(x.split('comic')[1].split('.')[0]))
data

['pp_comic1.md',
 'pp_comic2.md',
 'pp_comic3.md',
 'pp_comic4.md',
 'pp_comic5.md',
 'pp_comic6.md',
 'pp_comic7.md',
 'pp_comic8.md',
 'pp_comic9.md',
 'pp_comic10.md',
 'pp_comic11.md',
 'pp_comic12.md',
 'pp_comic13.md',
 'pp_comic14.md',
 'pp_comic15.md',
 'pp_comic16.md',
 'pp_comic17.md']

In [125]:
# Write this to the collection
new_doc.set({'content': data})

update_time {
  seconds: 1720974531
  nanos: 597079000
}

In [126]:
feed

{'2023-07-03 21:52:08': {'location': 'comics/pp_comic1.md'},
 '2023-07-04 12:53:11': {'location': 'comics/pp_comic2.md'},
 '2023-07-04 17:10:46': {'location': 'comics/pp_comic3.md'},
 '2023-07-06 07:23:12': {'location': 'comics/pp_comic4.md'},
 '2023-07-06 09:49:26': {'location': 'comics/pp_comic5.md'},
 '2023-07-06 14:10:46': {'location': 'comics/pp_comic6.md'},
 '2023-07-08 15:20:42': {'location': 'comics/pp_comic7.md'},
 '2023-07-09 14:28:02': {'location': 'comics/pp_comic8.md'},
 '2023-07-13 10:00:22': {'location': 'comics/pp_comic9.md'},
 '2023-07-18 13:16:07': {'location': 'comics/pp_comic10.md'},
 '2023-07-21 18:02:45': {'location': 'comics/pp_comic11.md'},
 '2023-07-22 16:33:22': {'location': 'comics/pp_comic12.md'},
 '2023-07-30 08:18:48': {'location': 'comics/pp_comic13.md'},
 '2023-12-11 10:10:20': {'location': 'comics/pp_comic14.md'},
 '2023-12-12 17:11:52': {'location': 'comics/pp_comic15.md'},
 '2023-12-26 14:03:03': {'location': 'comics/pp_comic16.md'},
 '2024-06-07 12:2

In [127]:
# Let's get the feed, update it with this and put it back
old_feed = db.collection('feed').document('content-log')
old_feed = old_feed.get().to_dict()
old_feed


{'2004-08-16 00:00:07': {'location': 'comics/hewligg_urobokkle_8.md'},
 '2024-04-23 08:17:04': {'location': 'blogs/internet.md'},
 '2004-08-23 00:00:00': {'location': 'comics/hewligg_urobokkle_21_3.md'},
 '2004-08-16 00:00:01': {'location': 'comics/hewligg_urobokkle_2.md'},
 '2024-02-01 17:08:24': {'location': 'blogs/hip.md'},
 '2004-10-15 00:00:00': {'location': 'comics/hewligg_urobokkle_38.md'},
 '2023-07-18 08:52:17': {'location': 'blogs/onrss.md'},
 '2024-01-04 11:35:56': {'location': 'blogs/books.md'},
 '2004-10-21 00:00:00': {'location': 'comics/hewligg_urobokkle_44.md'},
 '2004-08-17 00:00:03': {'location': 'comics/hewligg_urobokkle_13.md'},
 '2004-10-06 00:00:00': {'location': 'comics/hewligg_urobokkle_28.md'},
 '2004-10-27 00:00:00': {'location': 'comics/hewligg_urobokkle_50.md'},
 '2004-10-10 00:00:00': {'location': 'comics/hewligg_urobokkle_32.md'},
 '2004-08-18 00:00:01': {'location': 'comics/hewligg_urobokkle_17.md'},
 '2004-08-19 00:00:00': {'location': 'comics/hewligg_ur

In [128]:
# Update with the new feed
old_feed.update(feed)
old_feed

{'2004-08-16 00:00:07': {'location': 'comics/hewligg_urobokkle_8.md'},
 '2024-04-23 08:17:04': {'location': 'blogs/internet.md'},
 '2004-08-23 00:00:00': {'location': 'comics/hewligg_urobokkle_21_3.md'},
 '2004-08-16 00:00:01': {'location': 'comics/hewligg_urobokkle_2.md'},
 '2024-02-01 17:08:24': {'location': 'blogs/hip.md'},
 '2004-10-15 00:00:00': {'location': 'comics/hewligg_urobokkle_38.md'},
 '2023-07-18 08:52:17': {'location': 'blogs/onrss.md'},
 '2024-01-04 11:35:56': {'location': 'blogs/books.md'},
 '2004-10-21 00:00:00': {'location': 'comics/hewligg_urobokkle_44.md'},
 '2004-08-17 00:00:03': {'location': 'comics/hewligg_urobokkle_13.md'},
 '2004-10-06 00:00:00': {'location': 'comics/hewligg_urobokkle_28.md'},
 '2004-10-27 00:00:00': {'location': 'comics/hewligg_urobokkle_50.md'},
 '2004-10-10 00:00:00': {'location': 'comics/hewligg_urobokkle_32.md'},
 '2004-08-18 00:00:01': {'location': 'comics/hewligg_urobokkle_17.md'},
 '2004-08-19 00:00:00': {'location': 'comics/hewligg_ur

In [129]:
# Cool let's put it back
feed = db.collection('feed').document('content-log')
feed.set(old_feed)

update_time {
  seconds: 1720974593
  nanos: 988740000
}

In [130]:
# Sick now let's move staging to content
for root, dirs, files in os.walk('STAGING'):
    for file in files:
        shutil.move(os.path.join(root, file), os.path.join('CONTENT', root.split('/')[-1], file))
        print('Moved', file, 'to', os.path.join('CONTENT', root.split('/')[-1], file))

Moved pp_comic4.png to CONTENT/images/pp_comic4.png
Moved pp_comic_13_thumbnail.jpg to CONTENT/images/pp_comic_13_thumbnail.jpg
Moved pp_comic_8_thumbnail.jpg to CONTENT/images/pp_comic_8_thumbnail.jpg
Moved pp_comic_12_thumbnail.jpg to CONTENT/images/pp_comic_12_thumbnail.jpg
Moved pp_comic11.png to CONTENT/images/pp_comic11.png
Moved pp_comic_2_og.jpg to CONTENT/images/pp_comic_2_og.jpg
Moved pp_comic_5_thumbnail.jpg to CONTENT/images/pp_comic_5_thumbnail.jpg
Moved pp_comic14.png to CONTENT/images/pp_comic14.png
Moved pp_comic_13_og.jpg to CONTENT/images/pp_comic_13_og.jpg
Moved pp_comic_11_thumbnail.jpg to CONTENT/images/pp_comic_11_thumbnail.jpg
Moved pp_comic3.png to CONTENT/images/pp_comic3.png
Moved pp_comic16.png to CONTENT/images/pp_comic16.png
Moved pp_comic_1_thumbnail.jpg to CONTENT/images/pp_comic_1_thumbnail.jpg
Moved pp_comic_15_thumbnail.jpg to CONTENT/images/pp_comic_15_thumbnail.jpg
Moved pp_comic_16_thumbnail.jpg to CONTENT/images/pp_comic_16_thumbnail.jpg
Moved pp_c

In [131]:
from bs4 import BeautifulSoup
import requests
from markdownify import markdownify as md
# Time to do some scraping to get my coding heaven blog
homepage = 'https://codingheaven.btw.so/'

res = requests.get(homepage)
if res.status_code == 200:
    soup = BeautifulSoup(res.text, 'html.parser')
    # Get all the blog posts
    blog_posts = soup.find_all('a', class_='post-item')
    print('Found', len(blog_posts), 'blog posts')

Found 6 blog posts


In [139]:
from time import sleep
# Now let's request each blog post and get the content
for post in blog_posts:
    print('Requesting', post.get('href'))
    post_url = post.get('href')
    post_res = requests.get(post_url)
    if post_res.status_code == 200:
        post_soup = BeautifulSoup(post_res.text, 'html.parser')
        title = post_soup.find('h1').text
        date = post_soup.find(id='post-date-dd-mm-yyyy').text
        content = post_soup.find('article')
        img = content.find('img')
        content = md(str(content))
        # Request the image
        img_res = requests.get(img.get('src'))
        # Save to STAGING/images
        with open(f'STAGING/images/{title}.png', 'wb') as f:
            f.write(img_res.content)
        print('Got image', img.get('src'))
        # Construct markdown
        # We'll deal with thumbnails, og_image and description later
        markdown = f"""---
date: {date}
title: {title}
description: DESCRIPTION
author: Ed
tags: ['Coding Heaven', 'Blog']
type: blog
thumbnail: THUMBNAIL
og_title: {title}
og_description: DESCRIPTION
og_image: OG_IMAGE
og_type: article
collection: Coding Heaven
---
{content}
"""
        with open(f'STAGING/blogs/{title}.md', 'w') as f:
            f.write(markdown)
        print('Got content', title)
    else:
        print('Failed to request', post_url)
    print('Sleeping for 5 seconds')
    sleep(5)

Requesting https://codingheaven.btw.so/sql-queries-that-youll-never-need-but-should-try-anyway
Got image https://nyc3.digitaloceanspaces.com/btw-writer-prod/1719652767457%2FrecursiveCTE.png
Got content SQL Queries That You’ll Never Need (But Should Try Anyway)
Sleeping for 5 seconds
Requesting https://codingheaven.btw.so/making-an-image-crappifier
Got image https://nyc3.digitaloceanspaces.com/btw-writer-prod/1702983788357%2Fballdude.png
Got content Making an Image Crappifier
Sleeping for 5 seconds
Requesting https://codingheaven.btw.so/the-agony-of-bash-math
Got image https://nyc3.digitaloceanspaces.com/btw-writer-prod/1697641247914%2Fbashagony.png
Got content The Agony of Bash Math
Sleeping for 5 seconds
Requesting https://codingheaven.btw.so/fizz-buzz-but-its-excessively-overengineered
Got image https://nyc3.digitaloceanspaces.com/btw-writer-prod/1692982462664%2Ffizzbuzz.png
Got content Fizz Buzz but it’s Excessively Overengineered
Sleeping for 5 seconds
Requesting https://codingheav

In [142]:
# Since there's only 6 blogs we can just manually add the metadata and deal with the images
# Note the images in post are still linking to btw so we need to change that
# Also no time included in the date so we'll just add 00:00:00 boooo btw booooooo

# First things first lets find the descriptions for each tool and pop em in a list

homepage = 'https://codingheaven.btw.so/'
res = requests.get(homepage)
descriptions = []
if res.status_code == 200:
    soup = BeautifulSoup(res.text, 'html.parser')
    # Get all the blog posts
    blog_posts = soup.find_all('a', class_='post-item')
    # The description is the only p tag in the post-item
    for blog in blog_posts:
        descriptions.append(blog.find('p').text)

descriptions

['Let’s head to Codewars, a user-run site for creating and solving technical problems.',
 'You may read that title and wonder, why would anyone want to make an image crappifier?',
 'Let’s take a look at a coding problem that has plagued me for months on end:',
 "Ever since FizzBuzz first emerged in the famous piece 'Why Can't Programmers.. Program?', it's become a rite of passage. It's a simple program requiring bu...",
 "Humans count from 1, and yet arrays are indexed from 0 (unless you have the misfortune of using MATLAB). There's no wonder we suffer from the infamous off-by-one errors so oft...",
 'Hello World is usually the first program written by the budding programmer.']

In [143]:
# Let's link em to the md files
descriptions = {
    'bash_math.md': descriptions[2],
    'fizz_buzz.md': descriptions[3],
    'hello_world.md': descriptions[-1],
    'image_crappifier.md': descriptions[1],
    'off_by_one.md': descriptions[-2],
    'sql_injection.md': descriptions[0]
}

In [144]:
descriptions

{'bash_math.md': 'Let’s take a look at a coding problem that has plagued me for months on end:',
 'fizz_buzz.md': "Ever since FizzBuzz first emerged in the famous piece 'Why Can't Programmers.. Program?', it's become a rite of passage. It's a simple program requiring bu...",
 'hello_world.md': 'Hello World is usually the first program written by the budding programmer.',
 'image_crappifier.md': 'You may read that title and wonder, why would anyone want to make an image crappifier?',
 'off_by_one.md': "Humans count from 1, and yet arrays are indexed from 0 (unless you have the misfortune of using MATLAB). There's no wonder we suffer from the infamous off-by-one errors so oft...",
 'sql_injection.md': 'Let’s head to Codewars, a user-run site for creating and solving technical problems.'}

In [145]:
sql_recursive = descriptions.pop('sql_injection.md')
descriptions['sql_recursive.md'] = sql_recursive

In [157]:
import re
img_re = r'\!\[\]\((.*)\)'
# Cool now let's go update our md files
for f in os.listdir(os.path.join('STAGING', 'blogs')):
    # Split
    with open(os.path.join('STAGING', 'blogs', f), 'r') as file:
        md = file.read()

    split = md.split('---')

    metadata = split[1]
    content = '---'.join(split[2:])

    # Get rid of the first 4 lines in content - that's the title and other crap
    content = '\n'.join(content.split('\n')[4:])

    # Sub the image path into the markdown
    img = re.search(img_re, content)
    content = content.replace(img.group(1), f"/assets/images/{f.split('.')[0]}.png")

    # Sub the description
    metadata = metadata.replace('DESCRIPTION', descriptions[f])

    # Sub og_image
    metadata = metadata.replace('OG_IMAGE', f"/assets/images/{f.split('.')[0]}.png")

    # Create a thumbnail and sub the thumbnail
    img = Image.open(os.path.join('STAGING', 'images', f.split('.')[0] + '.png'))
    img.thumbnail((256, 256))
    img = img.convert('RGB')

    img.save(os.path.join('STAGING', 'images', f.split('.')[0] + '_thumbnail.jpg'))

    # Sub the thumbnail
    metadata = metadata.replace('THUMBNAIL', f"/assets/images/{f.split('.')[0]}_thumbnail.jpg")

    new_md = f"""---
{metadata.strip()}
---
{content.strip()}"""

    # Write it back
    with open(os.path.join('STAGING', 'blogs', f), 'w') as file:
        file.write(new_md)


In [158]:
# Cool now we can add these blogs to our feed
feed = db.collection('feed').document('content-log')
feed = feed.get().to_dict()

In [160]:
print('Feed length', len(feed))
for md_file in os.listdir(os.path.join('STAGING', 'blogs')):
    with open(os.path.join('STAGING', 'blogs', md_file), 'r') as f:
        md = f.read()

    # Get the date
    date = md.split('date: ')[1].split('\n')[0]
    date = dt.strptime(date, "%Y-%m-%d")
    while date.strftime("%Y-%m-%d %H:%M:%S") in feed:
        date = date + timedelta(seconds=1)

    feed[date.strftime("%Y-%m-%d %H:%M:%S")] = {
        'location': os.path.join('blogs', md_file)
    }

    print('Added', md_file)
    print('Time', date.strftime("%Y-%m-%d %H:%M:%S"))
    print('Location', os.path.join('blogs', md_file))

print('Feed length', len(feed))

Feed length 88
Added sql_recursive.md
Time 2024-06-29 00:00:00
Location blogs/sql_recursive.md
Added fizz_buzz.md
Time 2023-08-25 00:00:00
Location blogs/fizz_buzz.md
Added hello_world.md
Time 2023-07-29 00:00:00
Location blogs/hello_world.md
Added off_by_one.md
Time 2023-08-05 00:00:00
Location blogs/off_by_one.md
Added image_crappifier.md
Time 2023-12-19 00:00:00
Location blogs/image_crappifier.md
Added bash_math.md
Time 2023-10-18 00:00:00
Location blogs/bash_math.md
Feed length 94


In [161]:
feed

{'2023-07-06 14:10:46': {'location': 'comics/pp_comic6.md'},
 '2004-08-16 00:00:07': {'location': 'comics/hewligg_urobokkle_8.md'},
 '2024-04-23 08:17:04': {'location': 'blogs/internet.md'},
 '2004-08-23 00:00:00': {'location': 'comics/hewligg_urobokkle_21_3.md'},
 '2004-08-16 00:00:01': {'location': 'comics/hewligg_urobokkle_2.md'},
 '2024-02-01 17:08:24': {'location': 'blogs/hip.md'},
 '2004-10-15 00:00:00': {'location': 'comics/hewligg_urobokkle_38.md'},
 '2023-07-18 08:52:17': {'location': 'blogs/onrss.md'},
 '2024-01-04 11:35:56': {'location': 'blogs/books.md'},
 '2004-10-21 00:00:00': {'location': 'comics/hewligg_urobokkle_44.md'},
 '2004-08-17 00:00:03': {'location': 'comics/hewligg_urobokkle_13.md'},
 '2023-07-21 18:02:45': {'location': 'comics/pp_comic11.md'},
 '2023-07-09 14:28:02': {'location': 'comics/pp_comic8.md'},
 '2004-10-06 00:00:00': {'location': 'comics/hewligg_urobokkle_28.md'},
 '2023-07-18 13:16:07': {'location': 'comics/pp_comic10.md'},
 '2023-07-03 21:52:08': {

In [162]:
# Write it back
feed_ref = db.collection('feed').document('content-log')
feed_ref.set(feed)

update_time {
  seconds: 1721040246
  nanos: 49500000
}

In [164]:
# Oh we also need to write the collection
new_doc = db.collection('collections').document('coding_heaven')
content = [
    'hello_world.md',
    'off_by_one.md',
    'fizz_buzz.md',
    'bash_math.md',
    'image_crappifier.md',
    'sql_recursive.md'
]
content

['hello_world.md',
 'off_by_one.md',
 'fizz_buzz.md',
 'bash_math.md',
 'image_crappifier.md',
 'sql_recursive.md']

In [165]:
# Write it back
new_doc.set({'content': content})

update_time {
  seconds: 1721040452
  nanos: 997726000
}

In [166]:
# Issues with hello_world.md, let's open and save with utf-8 encoding
with open('CONTENT/blogs/hello_world.md', 'r', encoding='utf-8') as f:
    md = f.read()

with open('CONTENT/blogs/hello_world.md', 'w', encoding='utf-8') as f:
    f.write(md)

In [5]:
# Let's scrape weird indie shit
url = "https://weirdindieshit.blogspot.com/"

res = requests.get(url)
soup = BeautifulSoup(res.content, "html.parser")

In [7]:
post_titles = soup.find_all(name="h3", class_="post-title")
post_titles

[<h3 class="post-title"><a href="https://weirdindieshit.blogspot.com/2024/01/soviet-burgers-big-red-adventure.html">Soviet Burger's Big Red Adventure</a></h3>,
 <h3 class="post-title entry-title">
 <a href="https://weirdindieshit.blogspot.com/2023/12/mr-yuck-adventuers-in-jpeg-land.html">Mr Yuck Adventuers in JPEG Land</a>
 </h3>,
 <h3 class="post-title entry-title">
 <a href="https://weirdindieshit.blogspot.com/2023/08/work-at-mcdonald-authentic-fast-food.html">Work at McDonald, the Authentic Fast Food Worker Experience</a>
 </h3>,
 <h3 class="post-title entry-title">
 <a href="https://weirdindieshit.blogspot.com/2023/08/zapman-goes-on-slide-slide-into-chaos.html">Zapman Goes On a Slide, A Slide Into Chaos</a>
 </h3>,
 <h3 class="post-title entry-title">
 <a href="https://weirdindieshit.blogspot.com/2023/08/mouses-playground-affront-to-all-mice.html">Mouse's Playground, An Affront To All Mice Everywhere</a>
 </h3>,
 <h3 class="post-title entry-title">
 <a href="https://weirdindieshit.

In [10]:
# Start a dict
blogs = {
    title.find("a").text: title.find("a")['href']
    for title in post_titles
}
blogs

{"Soviet Burger's Big Red Adventure": 'https://weirdindieshit.blogspot.com/2024/01/soviet-burgers-big-red-adventure.html',
 'Mr Yuck Adventuers in JPEG Land': 'https://weirdindieshit.blogspot.com/2023/12/mr-yuck-adventuers-in-jpeg-land.html',
 'Work at McDonald, the Authentic Fast Food Worker Experience': 'https://weirdindieshit.blogspot.com/2023/08/work-at-mcdonald-authentic-fast-food.html',
 'Zapman Goes On a Slide, A Slide Into Chaos': 'https://weirdindieshit.blogspot.com/2023/08/zapman-goes-on-slide-slide-into-chaos.html',
 "Mouse's Playground, An Affront To All Mice Everywhere": 'https://weirdindieshit.blogspot.com/2023/08/mouses-playground-affront-to-all-mice.html',
 'FUCK HOUSE, When Bitsy Goes Weird': 'https://weirdindieshit.blogspot.com/2023/08/fuck-house-when-bitsy-goes-weird.html'}

In [12]:
from utils.string_utils import strip_punctuation
blog_content = {}
# Now let's make markdown files
for title, url in blogs.items():
    res = requests.get(url)
    blog_content[title] = res.content

blog_content

{"Soviet Burger's Big Red Adventure": b'<!DOCTYPE html>\n<html dir=\'ltr\' lang=\'en-GB\'>\n<head>\n<meta content=\'width=device-width, initial-scale=1\' name=\'viewport\'/>\n<title>Soviet Burger&#39;s Big Red Adventure</title>\n<meta content=\'text/html; charset=UTF-8\' http-equiv=\'Content-Type\'/>\n<!-- Chrome, Firefox OS and Opera -->\n<meta content=\'#eeeeee\' name=\'theme-color\'/>\n<!-- Windows Phone -->\n<meta content=\'#eeeeee\' name=\'msapplication-navbutton-color\'/>\n<meta content=\'blogger\' name=\'generator\'/>\n<link href=\'https://weirdindieshit.blogspot.com/favicon.ico\' rel=\'icon\' type=\'image/x-icon\'/>\n<link href=\'https://weirdindieshit.blogspot.com/2024/01/soviet-burgers-big-red-adventure.html\' rel=\'canonical\'/>\n<link rel="alternate" type="application/atom+xml" title="Weird Indie Shit - Atom" href="https://weirdindieshit.blogspot.com/feeds/posts/default" />\n<link rel="alternate" type="application/rss+xml" title="Weird Indie Shit - RSS" href="https://weirdi

In [19]:
# Well we need to download the images
for title, html in blog_content.items():
    soup = BeautifulSoup(html, 'html.parser')
    article = soup.find("article")
    for i, img in enumerate(article.find_all("img")):
        src = img['src']
        res = requests.get(src)
        image = res.content
        with open(os.path.join("STAGING", "images", strip_punctuation(title).replace(" ", "_") + str(i) + ".png"), "wb") as f:
            f.write(image)
        print("Got image", src)


Got image https://blogger.googleusercontent.com/img/b/R29vZ2xl/AVvXsEj5uCejHypuxrQy9YbCt6bdeuoLIM-99UrUQ9FFABxKut0Nr-RQBsZcBrqwQmswQZSdfhb-wybkAD4CSZUZurZaLWrzF9rBK011upLd9jsNyZad8GhpH8KaJgWQabDchlcfgPAn-WF9nSLTf5N_fmLZqUnNHuNYs8jZeLrtv3ZOzn6RaYd0BCPmU6ipTdlA/s16000/sovietburger2.png
Got image https://blogger.googleusercontent.com/img/b/R29vZ2xl/AVvXsEjTD_Z0cyZie48LE-gdjjZSvu5jxRrLhFkuPxvm6JPMs5Ad4k9h_x0vIINgDuwzSloMSNy5RIFH2YbvmdrgJOO_BpZK8TEge-F7hDAJYMb7db-s4BpIkVWrJYeoR7wYfB30xclx0aC7YQ92WRupl8ftaeYnoAi4qTkWKy9CB0O9z7sRP34Wv03LW4SKa5hu/s16000/sovietburger1.png
Got image https://blogger.googleusercontent.com/img/b/R29vZ2xl/AVvXsEgEpm7VwnNS9-PJIxzS26qlhtPxQQ6kxgQCJepmTlr2wTQmUBfAumx7faYRBA2c1xMbC6DyoRQb5hi-cNkKwzyBsdhUq91dETiRIQGjHHOevJiLXCNrugy0Gxb78EShS7j1_pWKlZwvWNnGveBtg-0QDnjju34qTI7UbqHLiWNtkOZOhdiaPj-m77pZdJJF/s16000/mryuck1.png
Got image https://blogger.googleusercontent.com/img/b/R29vZ2xl/AVvXsEhGH5amnJTU8r4FhJiosDKrgiOL0rF9bpVAjs3xLrxuIJwbYAD3tH8xoPG31DXinNbEvPzdBWFnZlcLiCvH

In [25]:
published_dts = {}
# Great let's start wrtiting our markdown docs
for title, html in blog_content.items():
    clean_title = strip_punctuation(title).replace(" ", "_")
    soup = BeautifulSoup(html, 'html.parser')
    article = soup.find("article")
    title = article.find("h3").text
    date_time = article.find('time')['datetime']
    date = date_time.split('T')[0]
    body = article.find("div", class_="post-body")
    post_content = md(str(body))
    description = body.find("p").text
    published_dts[clean_title] = date_time
    # Create a thumbnail
    img = Image.open(os.path.join("STAGING", "images", clean_title + "0.png"))
    img.thumbnail((256, 256))
    img = img.convert("RGB")
    img.save(os.path.join("STAGING", "images", clean_title + "_thumbnail.jpg"))

    with open(os.path.join("STAGING", "blogs", clean_title + ".md"), "w") as f:
        f.write(f"""---
date: {date}
title: {title}
description: {description}
author: Ed
tags: ['Video Game', 'Indie Game', 'Review']
type: blog
thumbnail: /assets/images/{clean_title}_thumbail.jpg
og_title: {title}
og_description: {description}
og_image: /asset/images/{clean_title}0.png
og_type: article
collection: Weird Indie Shit
---
{post_content}
""")
    print("Written blog", title)

Written blog 
Soviet Burger's Big Red Adventure

Written blog 
Mr Yuck Adventuers in JPEG Land

Written blog 
Work at McDonald, the Authentic Fast Food Worker Experience

Written blog 
Zapman Goes On a Slide, A Slide Into Chaos

Written blog 
Mouse's Playground, An Affront To All Mice Everywhere

Written blog 
FUCK HOUSE, When Bitsy Goes Weird



In [26]:
published_dts

{'Soviet_Burgers_Big_Red_Adventure': '2024-01-04T08:58:00-08:00',
 'Mr_Yuck_Adventuers_in_JPEG_Land': '2023-12-18T12:23:00-08:00',
 'Work_at_McDonald_the_Authentic_Fast_Food_Worker_Experience': '2023-08-22T05:47:00-07:00',
 'Zapman_Goes_On_a_Slide_A_Slide_Into_Chaos': '2023-08-14T23:49:00-07:00',
 'Mouses_Playground_An_Affront_To_All_Mice_Everywhere': '2023-08-08T01:50:00-07:00',
 'FUCK_HOUSE_When_Bitsy_Goes_Weird': '2023-08-05T03:23:00-07:00'}

In [34]:
# Now we can convert these to the correct format and put them in the feed
feed_ref = db.collection("collections").document("weird_indie_shit")
published_dts = {
    t+".md": _dt.replace("T", " ")
    for t, _dt in published_dts.items()
}
published_dts

{'Soviet_Burgers_Big_Red_Adventure.md': '2024-01-04 08:58:00-08:00',
 'Mr_Yuck_Adventuers_in_JPEG_Land.md': '2023-12-18 12:23:00-08:00',
 'Work_at_McDonald_the_Authentic_Fast_Food_Worker_Experience.md': '2023-08-22 05:47:00-07:00',
 'Zapman_Goes_On_a_Slide_A_Slide_Into_Chaos.md': '2023-08-14 23:49:00-07:00',
 'Mouses_Playground_An_Affront_To_All_Mice_Everywhere.md': '2023-08-08 01:50:00-07:00',
 'FUCK_HOUSE_When_Bitsy_Goes_Weird.md': '2023-08-05 03:23:00-07:00'}

In [43]:
# Includes timezones so let's loop over
for k, v in published_dts.items():
    hour_offset=v.split('-')[-1].split(':')[0]
    hour_offset = int(hour_offset)
    hour = v.split(' ')[1].split(':')[0]
    int_hour = int(hour)
    int_hour += hour_offset
    v = v.replace(f" {hour}", f" {str(int_hour)}")
    v = "-".join(v.split("-")[:-1])
    published_dts[k] = v

published_dts

{'Soviet_Burgers_Big_Red_Adventure.md': '2024-01-04 16:58:00',
 'Mr_Yuck_Adventuers_in_JPEG_Land.md': '2023-12-18 20:23:00',
 'Work_at_McDonald_the_Authentic_Fast_Food_Worker_Experience.md': '2023-08-22 12:47:00',
 'Zapman_Goes_On_a_Slide_A_Slide_Into_Chaos.md': '2023-08-14 30:49:00',
 'Mouses_Playground_An_Affront_To_All_Mice_Everywhere.md': '2023-08-08 8:50:00',
 'FUCK_HOUSE_When_Bitsy_Goes_Weird.md': '2023-08-05 10:23:00'}

In [31]:
# I deleted the feed so let's recover it fortunately I printed it out
feed = {'2023-07-06 14:10:46': {'location': 'comics/pp_comic6.md'},
 '2004-08-16 00:00:07': {'location': 'comics/hewligg_urobokkle_8.md'},
 '2024-04-23 08:17:04': {'location': 'blogs/internet.md'},
 '2004-08-23 00:00:00': {'location': 'comics/hewligg_urobokkle_21_3.md'},
 '2004-08-16 00:00:01': {'location': 'comics/hewligg_urobokkle_2.md'},
 '2024-02-01 17:08:24': {'location': 'blogs/hip.md'},
 '2004-10-15 00:00:00': {'location': 'comics/hewligg_urobokkle_38.md'},
 '2023-07-18 08:52:17': {'location': 'blogs/onrss.md'},
 '2024-01-04 11:35:56': {'location': 'blogs/books.md'},
 '2004-10-21 00:00:00': {'location': 'comics/hewligg_urobokkle_44.md'},
 '2004-08-17 00:00:03': {'location': 'comics/hewligg_urobokkle_13.md'},
 '2023-07-21 18:02:45': {'location': 'comics/pp_comic11.md'},
 '2023-07-09 14:28:02': {'location': 'comics/pp_comic8.md'},
 '2004-10-06 00:00:00': {'location': 'comics/hewligg_urobokkle_28.md'},
 '2023-07-18 13:16:07': {'location': 'comics/pp_comic10.md'},
 '2023-07-03 21:52:08': {'location': 'comics/pp_comic1.md'},
 '2004-10-27 00:00:00': {'location': 'comics/hewligg_urobokkle_50.md'},
 '2004-10-10 00:00:00': {'location': 'comics/hewligg_urobokkle_32.md'},
 '2004-08-18 00:00:01': {'location': 'comics/hewligg_urobokkle_17.md'},
 '2004-08-19 00:00:00': {'location': 'comics/hewligg_urobokkle_20_3.md'},
 '2023-02-22 00:00:00': {'location': 'blogs/stumbleupon.md'},
 '2004-10-04 00:00:00': {'location': 'comics/hewligg_urobokkle_26.md'},
 '2004-08-17 00:00:05': {'location': 'comics/hewligg_urobokkle_15.md'},
 '2004-08-17 00:00:04': {'location': 'comics/hewligg_urobokkle_14.md'},
 '2004-08-16 00:00:08': {'location': 'comics/hewligg_urobokkle_9.md'},
 '2004-09-30 00:00:00': {'location': 'comics/hewligg_urobokkle_22_2.md'},
 '2004-08-17 00:00:02': {'location': 'comics/hewligg_urobokkle_12.md'},
 '2004-10-14 00:00:00': {'location': 'comics/hewligg_urobokkle_37.md'},
 '2023-12-17 16:08:05': {'location': 'blogs/depressed.md'},
 '2004-10-24 00:00:00': {'location': 'comics/hewligg_urobokkle_47.md'},
 '2023-12-12 17:11:52': {'location': 'comics/pp_comic15.md'},
 '2023-07-22 16:33:22': {'location': 'comics/pp_comic12.md'},
 '2004-08-18 00:00:00': {'location': 'comics/hewligg_urobokkle_16.md'},
 '2004-08-16 00:00:00': {'location': 'comics/hewligg_urobokkle_1.md'},
 '2004-10-25 00:00:00': {'location': 'comics/hewligg_urobokkle_48.md'},
 '2004-08-16 00:00:06': {'location': 'comics/hewligg_urobokkle_7.md'},
 '2005-01-01 00:00:00': {'location': 'music/planet_ed.md'},
 '2024-05-06 09:38:07': {'location': 'blogs/discord.md'},
 '2004-10-07 00:00:00': {'location': 'comics/hewligg_urobokkle_29.md'},
 '2004-10-26 00:00:00': {'location': 'comics/hewligg_urobokkle_49.md'},
 '2004-10-09 00:00:00': {'location': 'comics/hewligg_urobokkle_31.md'},
 '2004-08-16 00:00:05': {'location': 'comics/hewligg_urobokkle_6.md'},
 '2023-07-23 10:22:07': {'location': 'blogs/burnout.md'},
 '2023-12-26 14:03:03': {'location': 'comics/pp_comic16.md'},
 '2004-10-16 00:00:00': {'location': 'comics/hewligg_urobokkle_39.md'},
 '2004-10-23 00:00:00': {'location': 'comics/hewligg_urobokkle_46.md'},
 '2024-03-14 00:00:00': {'location': 'blogs/horses.md'},
 '2004-08-17 00:00:01': {'location': 'comics/hewligg_urobokkle_11.md'},
 '2023-12-23 11:03:45': {'location': 'blogs/charlmes.md'},
 '2004-08-22 00:00:00': {'location': 'comics/hewligg_urobokkle_21_2.md'},
 '2004-10-11 00:00:00': {'location': 'comics/hewligg_urobokkle_33.md'},
 '2004-08-16 00:00:02': {'location': 'comics/hewligg_urobokkle_3.md'},
 '2004-08-16 00:00:04': {'location': 'comics/hewligg_urobokkle_5.md'},
 '2004-10-08 00:00:00': {'location': 'comics/hewligg_urobokkle_23.md'},
 '2023-07-13 10:00:22': {'location': 'comics/pp_comic9.md'},
 '2023-08-08 12:02:16': {'location': 'blogs/stress.md'},
 '2004-08-21 00:00:00': {'location': 'comics/hewligg_urobokkle_21_1.md'},
 '2004-08-18 00:00:03': {'location': 'comics/hewligg_urobokkle_19.md'},
 '2004-08-18 00:00:02': {'location': 'comics/hewligg_urobokkle_18.md'},
 '2004-08-16 00:00:03': {'location': 'comics/hewligg_urobokkle_4.md'},
 '2004-08-17 00:00:00': {'location': 'comics/hewligg_urobokkle_10.md'},
 '2004-10-19 00:00:00': {'location': 'comics/hewligg_urobokkle_42.md'},
 '2004-08-18 00:00:05': {'location': 'comics/hewligg_urobokkle_20_2.md'},
 '2004-10-13 00:00:00': {'location': 'comics/hewligg_urobokkle_36.md'},
 '2023-07-08 15:20:42': {'location': 'comics/pp_comic7.md'},
 '2004-10-20 00:00:00': {'location': 'comics/hewligg_urobokkle_43.md'},
 '2023-06-20 14:28:27': {'location': 'blogs/oldwebsites.md'},
 '2004-10-12 00:00:00': {'location': 'comics/hewligg_urobokkle_35.md'},
 '2004-10-02 00:00:00': {'location': 'comics/hewligg_urobokkle_24.md'},
 '2023-07-30 08:18:48': {'location': 'comics/pp_comic13.md'},
 '2023-04-17 00:00:00': {'location': 'blogs/bananas.md'},
 '2004-10-17 00:00:00': {'location': 'comics/hewligg_urobokkle_40.md'},
 '2023-07-29 09:05:47': {'location': 'blogs/dustydrawers.md'},
 '2023-07-06 09:49:26': {'location': 'comics/pp_comic5.md'},
 '2004-10-22 00:00:00': {'location': 'comics/hewligg_urobokkle_45.md'},
 '2023-07-06 07:23:12': {'location': 'comics/pp_comic4.md'},
 '2023-08-22 15:15:32': {'location': 'blogs/prank.md'},
 '2023-06-05 00:00:00': {'location': 'blogs/alcoholism.md'},
 '2004-10-18 00:00:00': {'location': 'comics/hewligg_urobokkle_41.md'},
 '2004-10-05 00:00:00': {'location': 'comics/hewligg_urobokkle_27.md'},
 '2023-07-04 17:10:46': {'location': 'comics/pp_comic3.md'},
 '2004-09-29 00:00:00': {'location': 'comics/hewligg_urobokkle_22_1.md'},
 '2004-10-08 00:00:01': {'location': 'comics/hewligg_urobokkle_30.md'},
 '2024-06-07 12:23:34': {'location': 'comics/pp_comic17.md'},
 '2023-07-04 12:53:11': {'location': 'comics/pp_comic2.md'},
 '2004-10-03 00:00:00': {'location': 'comics/hewligg_urobokkle_25.md'},
 '2004-08-18 00:00:04': {'location': 'comics/hewligg_urobokkle_20_1.md'},
 '2023-12-11 10:10:20': {'location': 'comics/pp_comic14.md'},
 '2024-06-29 00:00:00': {'location': 'blogs/sql_recursive.md'},
 '2023-08-25 00:00:00': {'location': 'blogs/fizz_buzz.md'},
 '2023-07-29 00:00:00': {'location': 'blogs/hello_world.md'},
 '2023-08-05 00:00:00': {'location': 'blogs/off_by_one.md'},
 '2023-12-19 00:00:00': {'location': 'blogs/image_crappifier.md'},
 '2023-10-18 00:00:00': {'location': 'blogs/bash_math.md'}}
feed

{'2023-07-06 14:10:46': {'location': 'comics/pp_comic6.md'},
 '2004-08-16 00:00:07': {'location': 'comics/hewligg_urobokkle_8.md'},
 '2024-04-23 08:17:04': {'location': 'blogs/internet.md'},
 '2004-08-23 00:00:00': {'location': 'comics/hewligg_urobokkle_21_3.md'},
 '2004-08-16 00:00:01': {'location': 'comics/hewligg_urobokkle_2.md'},
 '2024-02-01 17:08:24': {'location': 'blogs/hip.md'},
 '2004-10-15 00:00:00': {'location': 'comics/hewligg_urobokkle_38.md'},
 '2023-07-18 08:52:17': {'location': 'blogs/onrss.md'},
 '2024-01-04 11:35:56': {'location': 'blogs/books.md'},
 '2004-10-21 00:00:00': {'location': 'comics/hewligg_urobokkle_44.md'},
 '2004-08-17 00:00:03': {'location': 'comics/hewligg_urobokkle_13.md'},
 '2023-07-21 18:02:45': {'location': 'comics/pp_comic11.md'},
 '2023-07-09 14:28:02': {'location': 'comics/pp_comic8.md'},
 '2004-10-06 00:00:00': {'location': 'comics/hewligg_urobokkle_28.md'},
 '2023-07-18 13:16:07': {'location': 'comics/pp_comic10.md'},
 '2023-07-03 21:52:08': {

In [53]:
feed_ref = db.collection('feed').document('content-log')
feed = feed_ref.get().to_dict()
feed

{'2004-10-17 00:00:00': {'location': 'comics/hewligg_urobokkle_40.md'},
 '2023-07-06 14:10:46': {'location': 'comics/pp_comic6.md'},
 '2004-08-16 00:00:07': {'location': 'comics/hewligg_urobokkle_8.md'},
 '2024-04-23 08:17:04': {'location': 'blogs/internet.md'},
 '2004-08-23 00:00:00': {'location': 'comics/hewligg_urobokkle_21_3.md'},
 '2004-08-16 00:00:01': {'location': 'comics/hewligg_urobokkle_2.md'},
 '2024-02-01 17:08:24': {'location': 'blogs/hip.md'},
 '2004-10-15 00:00:00': {'location': 'comics/hewligg_urobokkle_38.md'},
 '2023-07-18 08:52:17': {'location': 'blogs/onrss.md'},
 '2024-01-04 11:35:56': {'location': 'blogs/books.md'},
 '2004-10-21 00:00:00': {'location': 'comics/hewligg_urobokkle_44.md'},
 '2004-08-17 00:00:03': {'location': 'comics/hewligg_urobokkle_13.md'},
 '2023-07-21 18:02:45': {'location': 'comics/pp_comic11.md'},
 '2023-07-09 14:28:02': {'location': 'comics/pp_comic8.md'},
 '2004-10-06 00:00:00': {'location': 'comics/hewligg_urobokkle_28.md'},
 '2023-07-18 13

In [47]:
# Now let's update the feed
new_items = {
    v: {'location': f'blogs/{k}'}
    for k, v in published_dts.items()
}
new_items

{'2024-01-04 16:58:00': {'location': 'blogs/Soviet_Burgers_Big_Red_Adventure.md'},
 '2023-12-18 20:23:00': {'location': 'blogs/Mr_Yuck_Adventuers_in_JPEG_Land.md'},
 '2023-08-22 12:47:00': {'location': 'blogs/Work_at_McDonald_the_Authentic_Fast_Food_Worker_Experience.md'},
 '2023-08-14 30:49:00': {'location': 'blogs/Zapman_Goes_On_a_Slide_A_Slide_Into_Chaos.md'},
 '2023-08-08 8:50:00': {'location': 'blogs/Mouses_Playground_An_Affront_To_All_Mice_Everywhere.md'},
 '2023-08-05 10:23:00': {'location': 'blogs/FUCK_HOUSE_When_Bitsy_Goes_Weird.md'}}

In [48]:
new_items['2023-08-08 08:50:00'] = new_items.pop('2023-08-08 8:50:00')
new_items

{'2024-01-04 16:58:00': {'location': 'blogs/Soviet_Burgers_Big_Red_Adventure.md'},
 '2023-12-18 20:23:00': {'location': 'blogs/Mr_Yuck_Adventuers_in_JPEG_Land.md'},
 '2023-08-22 12:47:00': {'location': 'blogs/Work_at_McDonald_the_Authentic_Fast_Food_Worker_Experience.md'},
 '2023-08-14 30:49:00': {'location': 'blogs/Zapman_Goes_On_a_Slide_A_Slide_Into_Chaos.md'},
 '2023-08-05 10:23:00': {'location': 'blogs/FUCK_HOUSE_When_Bitsy_Goes_Weird.md'},
 '2023-08-08 08:50:00': {'location': 'blogs/Mouses_Playground_An_Affront_To_All_Mice_Everywhere.md'}}

In [54]:
feed.update(new_items)
feed

{'2004-10-17 00:00:00': {'location': 'comics/hewligg_urobokkle_40.md'},
 '2023-07-06 14:10:46': {'location': 'comics/pp_comic6.md'},
 '2004-08-16 00:00:07': {'location': 'comics/hewligg_urobokkle_8.md'},
 '2024-04-23 08:17:04': {'location': 'blogs/internet.md'},
 '2004-08-23 00:00:00': {'location': 'comics/hewligg_urobokkle_21_3.md'},
 '2004-08-16 00:00:01': {'location': 'comics/hewligg_urobokkle_2.md'},
 '2024-02-01 17:08:24': {'location': 'blogs/hip.md'},
 '2004-10-15 00:00:00': {'location': 'comics/hewligg_urobokkle_38.md'},
 '2023-07-18 08:52:17': {'location': 'blogs/onrss.md'},
 '2024-01-04 11:35:56': {'location': 'blogs/books.md'},
 '2004-10-21 00:00:00': {'location': 'comics/hewligg_urobokkle_44.md'},
 '2004-08-17 00:00:03': {'location': 'comics/hewligg_urobokkle_13.md'},
 '2023-07-21 18:02:45': {'location': 'comics/pp_comic11.md'},
 '2023-07-09 14:28:02': {'location': 'comics/pp_comic8.md'},
 '2004-10-06 00:00:00': {'location': 'comics/hewligg_urobokkle_28.md'},
 '2023-07-18 13

In [55]:
# Write it back
feed_ref.set(feed)

update_time {
  seconds: 1721303500
  nanos: 31651000
}

In [56]:
# We need a list of conteent for the weird indie shit collection
doc_ref = db.collection('collections').document('weird_indie_shit')
content = [
    r.split('/')[-1] for r in published_dts.keys()
]
content

['Soviet_Burgers_Big_Red_Adventure.md',
 'Mr_Yuck_Adventuers_in_JPEG_Land.md',
 'Work_at_McDonald_the_Authentic_Fast_Food_Worker_Experience.md',
 'Zapman_Goes_On_a_Slide_A_Slide_Into_Chaos.md',
 'Mouses_Playground_An_Affront_To_All_Mice_Everywhere.md',
 'FUCK_HOUSE_When_Bitsy_Goes_Weird.md']

In [57]:
content.reverse()
content

['FUCK_HOUSE_When_Bitsy_Goes_Weird.md',
 'Mouses_Playground_An_Affront_To_All_Mice_Everywhere.md',
 'Zapman_Goes_On_a_Slide_A_Slide_Into_Chaos.md',
 'Work_at_McDonald_the_Authentic_Fast_Food_Worker_Experience.md',
 'Mr_Yuck_Adventuers_in_JPEG_Land.md',
 'Soviet_Burgers_Big_Red_Adventure.md']

In [58]:
# Put it up there
doc_ref.set({
    'content':
        content
})

update_time {
  seconds: 1721307072
  nanos: 580100000
}

In [60]:
vid_json = {
  "kind": "youtube#searchListResponse",
  "etag": "VJ2zqVD0q1s3H8-MFdmLfe-zcCE",
  "regionCode": "GB",
  "pageInfo": {
    "totalResults": 32,
    "resultsPerPage": 32
  },
  "items": [
    {
      "kind": "youtube#searchResult",
      "etag": "mSaOB8k-MFGZG_x9AmRl9vi48dg",
      "id": {
        "kind": "youtube#video",
        "videoId": "jXSt3af6Hj4"
      },
      "snippet": {
        "publishedAt": "2022-12-15T18:42:57Z",
        "channelId": "UCSdC_KL-it-m3Z-CK9VUUrg",
        "title": "Why don&#39;t we play our games anymore?",
        "description": "Do you have hundreds of games in Steam? Do you accumulate games with good intentions and then never actually play them?",
        "thumbnails": {
          "default": {
            "url": "https://i.ytimg.com/vi/jXSt3af6Hj4/default.jpg",
            "width": 120,
            "height": 90
          },
          "medium": {
            "url": "https://i.ytimg.com/vi/jXSt3af6Hj4/mqdefault.jpg",
            "width": 320,
            "height": 180
          },
          "high": {
            "url": "https://i.ytimg.com/vi/jXSt3af6Hj4/hqdefault.jpg",
            "width": 480,
            "height": 360
          }
        },
        "channelTitle": "Classic Ed",
        "liveBroadcastContent": "none",
        "publishTime": "2022-12-15T18:42:57Z"
      }
    },
    {
      "kind": "youtube#searchResult",
      "etag": "ZCIQoTjpC0FjfuKQFCu9uPf4vLs",
      "id": {
        "kind": "youtube#video",
        "videoId": "f7BGiZgN7W0"
      },
      "snippet": {
        "publishedAt": "2022-01-15T12:13:39Z",
        "channelId": "UCSdC_KL-it-m3Z-CK9VUUrg",
        "title": "This is Your Brain on Rollercoaster Tycoon",
        "description": "This Is Your Brain on RollerCoaster Tycoon I love Theme Park simulators, but none as much as RollerCoaster Tycoon. I've been ...",
        "thumbnails": {
          "default": {
            "url": "https://i.ytimg.com/vi/f7BGiZgN7W0/default.jpg",
            "width": 120,
            "height": 90
          },
          "medium": {
            "url": "https://i.ytimg.com/vi/f7BGiZgN7W0/mqdefault.jpg",
            "width": 320,
            "height": 180
          },
          "high": {
            "url": "https://i.ytimg.com/vi/f7BGiZgN7W0/hqdefault.jpg",
            "width": 480,
            "height": 360
          }
        },
        "channelTitle": "Classic Ed",
        "liveBroadcastContent": "none",
        "publishTime": "2022-01-15T12:13:39Z"
      }
    },
    {
      "kind": "youtube#searchResult",
      "etag": "gLBONazH1vawYdFj4-v_6UUyhf4",
      "id": {
        "kind": "youtube#video",
        "videoId": "q6GCkyXVloc"
      },
      "snippet": {
        "publishedAt": "2022-03-25T13:13:54Z",
        "channelId": "UCSdC_KL-it-m3Z-CK9VUUrg",
        "title": "Bringing Back the Dopefish Screensaver: A Wayback Machine Adventure",
        "description": "Dopefish is an enemy from Commander Keen Secret of the Oracle. Back in its hey day it made many appearances and featured ...",
        "thumbnails": {
          "default": {
            "url": "https://i.ytimg.com/vi/q6GCkyXVloc/default.jpg",
            "width": 120,
            "height": 90
          },
          "medium": {
            "url": "https://i.ytimg.com/vi/q6GCkyXVloc/mqdefault.jpg",
            "width": 320,
            "height": 180
          },
          "high": {
            "url": "https://i.ytimg.com/vi/q6GCkyXVloc/hqdefault.jpg",
            "width": 480,
            "height": 360
          }
        },
        "channelTitle": "Classic Ed",
        "liveBroadcastContent": "none",
        "publishTime": "2022-03-25T13:13:54Z"
      }
    },
    {
      "kind": "youtube#searchResult",
      "etag": "kDIyDSgZ3P41SPkRrsOSanzixqs",
      "id": {
        "kind": "youtube#video",
        "videoId": "kYYz9nXLpQM"
      },
      "snippet": {
        "publishedAt": "2023-07-02T11:24:15Z",
        "channelId": "UCSdC_KL-it-m3Z-CK9VUUrg",
        "title": "Commander Keen&#39;s Last Adventure, Unveiling the Lipton Tea Connection",
        "description": "Aliens Ate My Babysitter was the final title in the Commander Keen series of DOS games, and it didn't really do that well.",
        "thumbnails": {
          "default": {
            "url": "https://i.ytimg.com/vi/kYYz9nXLpQM/default.jpg",
            "width": 120,
            "height": 90
          },
          "medium": {
            "url": "https://i.ytimg.com/vi/kYYz9nXLpQM/mqdefault.jpg",
            "width": 320,
            "height": 180
          },
          "high": {
            "url": "https://i.ytimg.com/vi/kYYz9nXLpQM/hqdefault.jpg",
            "width": 480,
            "height": 360
          }
        },
        "channelTitle": "Classic Ed",
        "liveBroadcastContent": "none",
        "publishTime": "2023-07-02T11:24:15Z"
      }
    },
    {
      "kind": "youtube#searchResult",
      "etag": "PuO3fDWtZZ2gBKEJClWpiWmXa3M",
      "id": {
        "kind": "youtube#video",
        "videoId": "4mR7Biux-Hw"
      },
      "snippet": {
        "publishedAt": "2022-01-22T13:04:12Z",
        "channelId": "UCSdC_KL-it-m3Z-CK9VUUrg",
        "title": "An Unexpected Comment on Egyptian Theming Objects #shorts",
        "description": "I was looking at some RollerCoaster Tycoon sprites for making a thumbnail and came across a rather dramatic comment on ...",
        "thumbnails": {
          "default": {
            "url": "https://i.ytimg.com/vi/4mR7Biux-Hw/default.jpg",
            "width": 120,
            "height": 90
          },
          "medium": {
            "url": "https://i.ytimg.com/vi/4mR7Biux-Hw/mqdefault.jpg",
            "width": 320,
            "height": 180
          },
          "high": {
            "url": "https://i.ytimg.com/vi/4mR7Biux-Hw/hqdefault.jpg",
            "width": 480,
            "height": 360
          }
        },
        "channelTitle": "Classic Ed",
        "liveBroadcastContent": "none",
        "publishTime": "2022-01-22T13:04:12Z"
      }
    },
    {
      "kind": "youtube#searchResult",
      "etag": "mEfNvhM9CFPyQ-nRV5EImHM-6PY",
      "id": {
        "kind": "youtube#video",
        "videoId": "1Rlbqk9c8ro"
      },
      "snippet": {
        "publishedAt": "2022-05-14T12:33:00Z",
        "channelId": "UCSdC_KL-it-m3Z-CK9VUUrg",
        "title": "Why You Can&#39;t Play Fun School 6 Anymore",
        "description": "FunSchool 6, Magicland is a game from my childhood that I've wanted to play again for years. After a lot of difficulty finding it, ...",
        "thumbnails": {
          "default": {
            "url": "https://i.ytimg.com/vi/1Rlbqk9c8ro/default.jpg",
            "width": 120,
            "height": 90
          },
          "medium": {
            "url": "https://i.ytimg.com/vi/1Rlbqk9c8ro/mqdefault.jpg",
            "width": 320,
            "height": 180
          },
          "high": {
            "url": "https://i.ytimg.com/vi/1Rlbqk9c8ro/hqdefault.jpg",
            "width": 480,
            "height": 360
          }
        },
        "channelTitle": "Classic Ed",
        "liveBroadcastContent": "none",
        "publishTime": "2022-05-14T12:33:00Z"
      }
    },
    {
      "kind": "youtube#searchResult",
      "etag": "txtlp_q_Nad_xNRuLpqHtRD8qIQ",
      "id": {
        "kind": "youtube#video",
        "videoId": "W4XYlp8ZBMg"
      },
      "snippet": {
        "publishedAt": "2022-11-03T11:37:50Z",
        "channelId": "UCSdC_KL-it-m3Z-CK9VUUrg",
        "title": "Ratchet and Clank Nostalgia Video",
        "description": "Ratchet and Clank memories and nostalgia and whatnot.",
        "thumbnails": {
          "default": {
            "url": "https://i.ytimg.com/vi/W4XYlp8ZBMg/default.jpg",
            "width": 120,
            "height": 90
          },
          "medium": {
            "url": "https://i.ytimg.com/vi/W4XYlp8ZBMg/mqdefault.jpg",
            "width": 320,
            "height": 180
          },
          "high": {
            "url": "https://i.ytimg.com/vi/W4XYlp8ZBMg/hqdefault.jpg",
            "width": 480,
            "height": 360
          }
        },
        "channelTitle": "Classic Ed",
        "liveBroadcastContent": "none",
        "publishTime": "2022-11-03T11:37:50Z"
      }
    },
    {
      "kind": "youtube#searchResult",
      "etag": "ZIGskP4YJGtJYWLHBkAb3lMdqjw",
      "id": {
        "kind": "youtube#video",
        "videoId": "xzKzg_bJ0DU"
      },
      "snippet": {
        "publishedAt": "2022-05-21T12:34:14Z",
        "channelId": "UCSdC_KL-it-m3Z-CK9VUUrg",
        "title": "The Worst Box in Crash 2",
        "description": "Crash Bandicoot 2: Cortex Strikes Back is a pretty good game...apart from Cold Hard Crash. That level sucks because of the box ...",
        "thumbnails": {
          "default": {
            "url": "https://i.ytimg.com/vi/xzKzg_bJ0DU/default.jpg",
            "width": 120,
            "height": 90
          },
          "medium": {
            "url": "https://i.ytimg.com/vi/xzKzg_bJ0DU/mqdefault.jpg",
            "width": 320,
            "height": 180
          },
          "high": {
            "url": "https://i.ytimg.com/vi/xzKzg_bJ0DU/hqdefault.jpg",
            "width": 480,
            "height": 360
          }
        },
        "channelTitle": "Classic Ed",
        "liveBroadcastContent": "none",
        "publishTime": "2022-05-21T12:34:14Z"
      }
    },
    {
      "kind": "youtube#searchResult",
      "etag": "5XH3O84ldWcBDJHDtxJBdZSaZgU",
      "id": {
        "kind": "youtube#video",
        "videoId": "64lHWNqK2jg"
      },
      "snippet": {
        "publishedAt": "2024-02-12T00:00:01Z",
        "channelId": "UCSdC_KL-it-m3Z-CK9VUUrg",
        "title": "The People Who Make Bad Games",
        "description": "Making bad games on purpose Thanks to my friend Adrien for playing lots of terrible games so I can use his footage and not have ...",
        "thumbnails": {
          "default": {
            "url": "https://i.ytimg.com/vi/64lHWNqK2jg/default.jpg",
            "width": 120,
            "height": 90
          },
          "medium": {
            "url": "https://i.ytimg.com/vi/64lHWNqK2jg/mqdefault.jpg",
            "width": 320,
            "height": 180
          },
          "high": {
            "url": "https://i.ytimg.com/vi/64lHWNqK2jg/hqdefault.jpg",
            "width": 480,
            "height": 360
          }
        },
        "channelTitle": "Classic Ed",
        "liveBroadcastContent": "none",
        "publishTime": "2024-02-12T00:00:01Z"
      }
    },
    {
      "kind": "youtube#searchResult",
      "etag": "iWmmCLjVhQqj2ELdscp6Z8yvtZQ",
      "id": {
        "kind": "youtube#video",
        "videoId": "S-OiTywWWnM"
      },
      "snippet": {
        "publishedAt": "2022-06-29T16:07:45Z",
        "channelId": "UCSdC_KL-it-m3Z-CK9VUUrg",
        "title": "Dunnet - The Secret Terminal Game",
        "description": "Terminal is a powerful tool that we interact with every day. It's the shell interface to interacting with our computer. But what if there ...",
        "thumbnails": {
          "default": {
            "url": "https://i.ytimg.com/vi/S-OiTywWWnM/default.jpg",
            "width": 120,
            "height": 90
          },
          "medium": {
            "url": "https://i.ytimg.com/vi/S-OiTywWWnM/mqdefault.jpg",
            "width": 320,
            "height": 180
          },
          "high": {
            "url": "https://i.ytimg.com/vi/S-OiTywWWnM/hqdefault.jpg",
            "width": 480,
            "height": 360
          }
        },
        "channelTitle": "Classic Ed",
        "liveBroadcastContent": "none",
        "publishTime": "2022-06-29T16:07:45Z"
      }
    },
    {
      "kind": "youtube#searchResult",
      "etag": "dkI0DGZ2scsxac-EWUVmtwoHDuc",
      "id": {
        "kind": "youtube#video",
        "videoId": "JLA0sZ5xDYw"
      },
      "snippet": {
        "publishedAt": "2023-12-14T19:58:28Z",
        "channelId": "UCSdC_KL-it-m3Z-CK9VUUrg",
        "title": "Reject Social Media, Return to RSS",
        "description": "RSS is cool.",
        "thumbnails": {
          "default": {
            "url": "https://i.ytimg.com/vi/JLA0sZ5xDYw/default.jpg",
            "width": 120,
            "height": 90
          },
          "medium": {
            "url": "https://i.ytimg.com/vi/JLA0sZ5xDYw/mqdefault.jpg",
            "width": 320,
            "height": 180
          },
          "high": {
            "url": "https://i.ytimg.com/vi/JLA0sZ5xDYw/hqdefault.jpg",
            "width": 480,
            "height": 360
          }
        },
        "channelTitle": "Classic Ed",
        "liveBroadcastContent": "none",
        "publishTime": "2023-12-14T19:58:28Z"
      }
    },
    {
      "kind": "youtube#searchResult",
      "etag": "F4z69rhu_ffwEUMte-gQzj-LjmM",
      "id": {
        "kind": "youtube#video",
        "videoId": "5DB7u4YMY1I"
      },
      "snippet": {
        "publishedAt": "2022-03-08T19:36:04Z",
        "channelId": "UCSdC_KL-it-m3Z-CK9VUUrg",
        "title": "The Games That Nobody Plays",
        "description": "The Games That Nobody Plays We were hit by the indiepocalypse when Steam opened up their platform for everyone. All you ...",
        "thumbnails": {
          "default": {
            "url": "https://i.ytimg.com/vi/5DB7u4YMY1I/default.jpg",
            "width": 120,
            "height": 90
          },
          "medium": {
            "url": "https://i.ytimg.com/vi/5DB7u4YMY1I/mqdefault.jpg",
            "width": 320,
            "height": 180
          },
          "high": {
            "url": "https://i.ytimg.com/vi/5DB7u4YMY1I/hqdefault.jpg",
            "width": 480,
            "height": 360
          }
        },
        "channelTitle": "Classic Ed",
        "liveBroadcastContent": "none",
        "publishTime": "2022-03-08T19:36:04Z"
      }
    },
    {
      "kind": "youtube#searchResult",
      "etag": "7Ww7DjnJ00HhmhfIkCkFgF5b23c",
      "id": {
        "kind": "youtube#video",
        "videoId": "PxilnA6XBX4"
      },
      "snippet": {
        "publishedAt": "2022-02-20T12:00:23Z",
        "channelId": "UCSdC_KL-it-m3Z-CK9VUUrg",
        "title": "How Slime Volleyball Became a Cult Classic",
        "description": "Slime Volleyball is a cult classic game and has had thousands upon thousands of players through the years. The game we've all ...",
        "thumbnails": {
          "default": {
            "url": "https://i.ytimg.com/vi/PxilnA6XBX4/default.jpg",
            "width": 120,
            "height": 90
          },
          "medium": {
            "url": "https://i.ytimg.com/vi/PxilnA6XBX4/mqdefault.jpg",
            "width": 320,
            "height": 180
          },
          "high": {
            "url": "https://i.ytimg.com/vi/PxilnA6XBX4/hqdefault.jpg",
            "width": 480,
            "height": 360
          }
        },
        "channelTitle": "Classic Ed",
        "liveBroadcastContent": "none",
        "publishTime": "2022-02-20T12:00:23Z"
      }
    },
    {
      "kind": "youtube#searchResult",
      "etag": "mxzidg7SkoYSmsUlBskEAX92fxI",
      "id": {
        "kind": "youtube#video",
        "videoId": "sRb-QiR9OVk"
      },
      "snippet": {
        "publishedAt": "2024-03-13T00:00:26Z",
        "channelId": "UCSdC_KL-it-m3Z-CK9VUUrg",
        "title": "A Game Too Obscure to Beat",
        "description": "Ever played a game and can't work out what to do but you also can't find anyone on the internet who knows what to do either?",
        "thumbnails": {
          "default": {
            "url": "https://i.ytimg.com/vi/sRb-QiR9OVk/default.jpg",
            "width": 120,
            "height": 90
          },
          "medium": {
            "url": "https://i.ytimg.com/vi/sRb-QiR9OVk/mqdefault.jpg",
            "width": 320,
            "height": 180
          },
          "high": {
            "url": "https://i.ytimg.com/vi/sRb-QiR9OVk/hqdefault.jpg",
            "width": 480,
            "height": 360
          }
        },
        "channelTitle": "Classic Ed",
        "liveBroadcastContent": "none",
        "publishTime": "2024-03-13T00:00:26Z"
      }
    },
    {
      "kind": "youtube#searchResult",
      "etag": "KnE_bYeM0vDKZ7f5-6Esh2uu2cs",
      "id": {
        "kind": "youtube#video",
        "videoId": "dkde76rDEjw"
      },
      "snippet": {
        "publishedAt": "2022-07-28T15:44:13Z",
        "channelId": "UCSdC_KL-it-m3Z-CK9VUUrg",
        "title": "Sonic&#39;s Blunder Years",
        "description": "The Game Gear library of Sonic games was surprisingly vast, and amongst those games was none other than Sonic Blast.",
        "thumbnails": {
          "default": {
            "url": "https://i.ytimg.com/vi/dkde76rDEjw/default.jpg",
            "width": 120,
            "height": 90
          },
          "medium": {
            "url": "https://i.ytimg.com/vi/dkde76rDEjw/mqdefault.jpg",
            "width": 320,
            "height": 180
          },
          "high": {
            "url": "https://i.ytimg.com/vi/dkde76rDEjw/hqdefault.jpg",
            "width": 480,
            "height": 360
          }
        },
        "channelTitle": "Classic Ed",
        "liveBroadcastContent": "none",
        "publishTime": "2022-07-28T15:44:13Z"
      }
    },
    {
      "kind": "youtube#searchResult",
      "etag": "I2rdornq_DAFItnY9--q5LZ1Ovc",
      "id": {
        "kind": "youtube#video",
        "videoId": "3IGs0rxhHVo"
      },
      "snippet": {
        "publishedAt": "2024-04-15T06:35:07Z",
        "channelId": "UCSdC_KL-it-m3Z-CK9VUUrg",
        "title": "I waited nearly 30 years to beat this game",
        "description": "hello.",
        "thumbnails": {
          "default": {
            "url": "https://i.ytimg.com/vi/3IGs0rxhHVo/default.jpg",
            "width": 120,
            "height": 90
          },
          "medium": {
            "url": "https://i.ytimg.com/vi/3IGs0rxhHVo/mqdefault.jpg",
            "width": 320,
            "height": 180
          },
          "high": {
            "url": "https://i.ytimg.com/vi/3IGs0rxhHVo/hqdefault.jpg",
            "width": 480,
            "height": 360
          }
        },
        "channelTitle": "Classic Ed",
        "liveBroadcastContent": "none",
        "publishTime": "2024-04-15T06:35:07Z"
      }
    },
    {
      "kind": "youtube#searchResult",
      "etag": "s45ez8YW_RF5Cq4UVywAUUrHsd8",
      "id": {
        "kind": "youtube#video",
        "videoId": "fzM1Gl6IWB0"
      },
      "snippet": {
        "publishedAt": "2022-01-03T15:27:21Z",
        "channelId": "UCSdC_KL-it-m3Z-CK9VUUrg",
        "title": "Doom but with Commander Keen Music #shorts",
        "description": "What if Bobby Prince wrote the soundtrack to Doom? Oh, he did?",
        "thumbnails": {
          "default": {
            "url": "https://i.ytimg.com/vi/fzM1Gl6IWB0/default.jpg",
            "width": 120,
            "height": 90
          },
          "medium": {
            "url": "https://i.ytimg.com/vi/fzM1Gl6IWB0/mqdefault.jpg",
            "width": 320,
            "height": 180
          },
          "high": {
            "url": "https://i.ytimg.com/vi/fzM1Gl6IWB0/hqdefault.jpg",
            "width": 480,
            "height": 360
          }
        },
        "channelTitle": "Classic Ed",
        "liveBroadcastContent": "none",
        "publishTime": "2022-01-03T15:27:21Z"
      }
    },
    {
      "kind": "youtube#searchResult",
      "etag": "kYs68hyR0gbjoxuSEOdegcdI5d8",
      "id": {
        "kind": "youtube#video",
        "videoId": "yX1OUtk50Wk"
      },
      "snippet": {
        "publishedAt": "2023-12-28T15:56:55Z",
        "channelId": "UCSdC_KL-it-m3Z-CK9VUUrg",
        "title": "Gaming Through Darkness",
        "description": "Playing games to help you through the dark times.",
        "thumbnails": {
          "default": {
            "url": "https://i.ytimg.com/vi/yX1OUtk50Wk/default.jpg",
            "width": 120,
            "height": 90
          },
          "medium": {
            "url": "https://i.ytimg.com/vi/yX1OUtk50Wk/mqdefault.jpg",
            "width": 320,
            "height": 180
          },
          "high": {
            "url": "https://i.ytimg.com/vi/yX1OUtk50Wk/hqdefault.jpg",
            "width": 480,
            "height": 360
          }
        },
        "channelTitle": "Classic Ed",
        "liveBroadcastContent": "none",
        "publishTime": "2023-12-28T15:56:55Z"
      }
    },
    {
      "kind": "youtube#searchResult",
      "etag": "ASpO6_t6Qpw277_pURRMY-Nwyew",
      "id": {
        "kind": "youtube#video",
        "videoId": "WEEO3yyBb0U"
      },
      "snippet": {
        "publishedAt": "2022-03-18T11:20:30Z",
        "channelId": "UCSdC_KL-it-m3Z-CK9VUUrg",
        "title": "A Closer Look at the Rare 666 Edition of Slime Volleyball",
        "description": "In my Brief History of Slime Volleyball video I mentioned that the 666 edition of One Slime had disappeared off the internet.",
        "thumbnails": {
          "default": {
            "url": "https://i.ytimg.com/vi/WEEO3yyBb0U/default.jpg",
            "width": 120,
            "height": 90
          },
          "medium": {
            "url": "https://i.ytimg.com/vi/WEEO3yyBb0U/mqdefault.jpg",
            "width": 320,
            "height": 180
          },
          "high": {
            "url": "https://i.ytimg.com/vi/WEEO3yyBb0U/hqdefault.jpg",
            "width": 480,
            "height": 360
          }
        },
        "channelTitle": "Classic Ed",
        "liveBroadcastContent": "none",
        "publishTime": "2022-03-18T11:20:30Z"
      }
    },
    {
      "kind": "youtube#searchResult",
      "etag": "RFGTT4FuGH3QxPjUEJ0e97tAMvo",
      "id": {
        "kind": "youtube#video",
        "videoId": "t3ShM5Zf694"
      },
      "snippet": {
        "publishedAt": "2022-10-11T18:29:51Z",
        "channelId": "UCSdC_KL-it-m3Z-CK9VUUrg",
        "title": "How Fans Resurrected Jak and Daxter in 2022",
        "description": "Jak and Daxter finally gets some new developments after years of neglect thanks to its fanbase. Can't be bothered to write a good ...",
        "thumbnails": {
          "default": {
            "url": "https://i.ytimg.com/vi/t3ShM5Zf694/default.jpg",
            "width": 120,
            "height": 90
          },
          "medium": {
            "url": "https://i.ytimg.com/vi/t3ShM5Zf694/mqdefault.jpg",
            "width": 320,
            "height": 180
          },
          "high": {
            "url": "https://i.ytimg.com/vi/t3ShM5Zf694/hqdefault.jpg",
            "width": 480,
            "height": 360
          }
        },
        "channelTitle": "Classic Ed",
        "liveBroadcastContent": "none",
        "publishTime": "2022-10-11T18:29:51Z"
      }
    },
    {
      "kind": "youtube#searchResult",
      "etag": "rfh2qBAURtxKgCdR6dWf_b33ObQ",
      "id": {
        "kind": "youtube#video",
        "videoId": "79iDJfCXrJc"
      },
      "snippet": {
        "publishedAt": "2024-05-14T09:28:11Z",
        "channelId": "UCSdC_KL-it-m3Z-CK9VUUrg",
        "title": "Ted Crusty, a lost legend of Youtube",
        "description": "",
        "thumbnails": {
          "default": {
            "url": "https://i.ytimg.com/vi/79iDJfCXrJc/default.jpg",
            "width": 120,
            "height": 90
          },
          "medium": {
            "url": "https://i.ytimg.com/vi/79iDJfCXrJc/mqdefault.jpg",
            "width": 320,
            "height": 180
          },
          "high": {
            "url": "https://i.ytimg.com/vi/79iDJfCXrJc/hqdefault.jpg",
            "width": 480,
            "height": 360
          }
        },
        "channelTitle": "Classic Ed",
        "liveBroadcastContent": "none",
        "publishTime": "2024-05-14T09:28:11Z"
      }
    },
    {
      "kind": "youtube#searchResult",
      "etag": "hepVKKadvJUrkuBi97p8o1S73RE",
      "id": {
        "kind": "youtube#video",
        "videoId": "7xE5iwG6EO0"
      },
      "snippet": {
        "publishedAt": "2022-02-02T15:00:07Z",
        "channelId": "UCSdC_KL-it-m3Z-CK9VUUrg",
        "title": "The Slime Volley Ball Experience #shorts",
        "description": "Slime Volleyball, aka One Slime, is actually a mod of a 2-player remake of this really old game called Slime Volley Ball. I played it.",
        "thumbnails": {
          "default": {
            "url": "https://i.ytimg.com/vi/7xE5iwG6EO0/default.jpg",
            "width": 120,
            "height": 90
          },
          "medium": {
            "url": "https://i.ytimg.com/vi/7xE5iwG6EO0/mqdefault.jpg",
            "width": 320,
            "height": 180
          },
          "high": {
            "url": "https://i.ytimg.com/vi/7xE5iwG6EO0/hqdefault.jpg",
            "width": 480,
            "height": 360
          }
        },
        "channelTitle": "Classic Ed",
        "liveBroadcastContent": "none",
        "publishTime": "2022-02-02T15:00:07Z"
      }
    },
    {
      "kind": "youtube#searchResult",
      "etag": "DKY3T6TUVLXWprcAl_f_b_7_eVY",
      "id": {
        "kind": "youtube#video",
        "videoId": "5Sy7e-pz2c8"
      },
      "snippet": {
        "publishedAt": "2022-06-08T20:08:33Z",
        "channelId": "UCSdC_KL-it-m3Z-CK9VUUrg",
        "title": "StumbleUpon: Internet Discovery Through the 00&#39;s",
        "description": "StumbleUpon was a discovery engine toolbar/browser extension and eventually an app that got you stumbling upon random cool ...",
        "thumbnails": {
          "default": {
            "url": "https://i.ytimg.com/vi/5Sy7e-pz2c8/default.jpg",
            "width": 120,
            "height": 90
          },
          "medium": {
            "url": "https://i.ytimg.com/vi/5Sy7e-pz2c8/mqdefault.jpg",
            "width": 320,
            "height": 180
          },
          "high": {
            "url": "https://i.ytimg.com/vi/5Sy7e-pz2c8/hqdefault.jpg",
            "width": 480,
            "height": 360
          }
        },
        "channelTitle": "Classic Ed",
        "liveBroadcastContent": "none",
        "publishTime": "2022-06-08T20:08:33Z"
      }
    },
    {
      "kind": "youtube#searchResult",
      "etag": "AcR0zNDc9irEUoPD3ABCKP16Zrg",
      "id": {
        "kind": "youtube#video",
        "videoId": "WTiLdxBUayI"
      },
      "snippet": {
        "publishedAt": "2022-01-26T19:00:11Z",
        "channelId": "UCSdC_KL-it-m3Z-CK9VUUrg",
        "title": "A Commander Keen 5 Easter Egg and...wait, what? #shorts",
        "description": "Commander Keen episode 5: The Armageddon Machine has an easter egg hidden in Energy Flow Systems but I got more than I ...",
        "thumbnails": {
          "default": {
            "url": "https://i.ytimg.com/vi/WTiLdxBUayI/default.jpg",
            "width": 120,
            "height": 90
          },
          "medium": {
            "url": "https://i.ytimg.com/vi/WTiLdxBUayI/mqdefault.jpg",
            "width": 320,
            "height": 180
          },
          "high": {
            "url": "https://i.ytimg.com/vi/WTiLdxBUayI/hqdefault.jpg",
            "width": 480,
            "height": 360
          }
        },
        "channelTitle": "Classic Ed",
        "liveBroadcastContent": "none",
        "publishTime": "2022-01-26T19:00:11Z"
      }
    },
    {
      "kind": "youtube#searchResult",
      "etag": "mj9quqj8fj5XEVdI6C5fQNpcUZY",
      "id": {
        "kind": "youtube#video",
        "videoId": "OvZIoH8d_AY"
      },
      "snippet": {
        "publishedAt": "2023-06-12T18:57:29Z",
        "channelId": "UCSdC_KL-it-m3Z-CK9VUUrg",
        "title": "Nintendo Nightmare is Illegal, Here&#39;s Why",
        "description": "In this video I talk about the early indie development community, inparticular the community that emerged when Gamemaker 6 ...",
        "thumbnails": {
          "default": {
            "url": "https://i.ytimg.com/vi/OvZIoH8d_AY/default.jpg",
            "width": 120,
            "height": 90
          },
          "medium": {
            "url": "https://i.ytimg.com/vi/OvZIoH8d_AY/mqdefault.jpg",
            "width": 320,
            "height": 180
          },
          "high": {
            "url": "https://i.ytimg.com/vi/OvZIoH8d_AY/hqdefault.jpg",
            "width": 480,
            "height": 360
          }
        },
        "channelTitle": "Classic Ed",
        "liveBroadcastContent": "none",
        "publishTime": "2023-06-12T18:57:29Z"
      }
    },
    {
      "kind": "youtube#searchResult",
      "etag": "zzZm6mXA1XU9I1bV49OZqARlSAM",
      "id": {
        "kind": "youtube#video",
        "videoId": "BOeW7F8sRNE"
      },
      "snippet": {
        "publishedAt": "2022-08-30T23:00:03Z",
        "channelId": "UCSdC_KL-it-m3Z-CK9VUUrg",
        "title": "This Ratchet and Clank Game was Lost for 20 Years",
        "description": "This is a video about the Ratchet and Clank 2002 press kit CD-ROM, which contained a bunch of stuff, including a really weird ...",
        "thumbnails": {
          "default": {
            "url": "https://i.ytimg.com/vi/BOeW7F8sRNE/default.jpg",
            "width": 120,
            "height": 90
          },
          "medium": {
            "url": "https://i.ytimg.com/vi/BOeW7F8sRNE/mqdefault.jpg",
            "width": 320,
            "height": 180
          },
          "high": {
            "url": "https://i.ytimg.com/vi/BOeW7F8sRNE/hqdefault.jpg",
            "width": 480,
            "height": 360
          }
        },
        "channelTitle": "Classic Ed",
        "liveBroadcastContent": "none",
        "publishTime": "2022-08-30T23:00:03Z"
      }
    },
    {
      "kind": "youtube#searchResult",
      "etag": "TEUnrN6Hml_pXi_JnA2xBY19uWU",
      "id": {
        "kind": "youtube#video",
        "videoId": "iQQkInVetS4"
      },
      "snippet": {
        "publishedAt": "2021-12-26T20:06:08Z",
        "channelId": "UCSdC_KL-it-m3Z-CK9VUUrg",
        "title": "How Commander Keen Changed the Face of DOS Gaming Forever",
        "description": "Part 2 here: https://youtu.be/7kBIu_4IBrM Commander Keen is a classic game from the shareware era of DOS games, but it was ...",
        "thumbnails": {
          "default": {
            "url": "https://i.ytimg.com/vi/iQQkInVetS4/default.jpg",
            "width": 120,
            "height": 90
          },
          "medium": {
            "url": "https://i.ytimg.com/vi/iQQkInVetS4/mqdefault.jpg",
            "width": 320,
            "height": 180
          },
          "high": {
            "url": "https://i.ytimg.com/vi/iQQkInVetS4/hqdefault.jpg",
            "width": 480,
            "height": 360
          }
        },
        "channelTitle": "Classic Ed",
        "liveBroadcastContent": "none",
        "publishTime": "2021-12-26T20:06:08Z"
      }
    },
    {
      "kind": "youtube#searchResult",
      "etag": "vUtsK-SiDKaB4d-ZeIjA7pXrn4c",
      "id": {
        "kind": "youtube#video",
        "videoId": "7kBIu_4IBrM"
      },
      "snippet": {
        "publishedAt": "2022-01-29T13:00:06Z",
        "channelId": "UCSdC_KL-it-m3Z-CK9VUUrg",
        "title": "Commander Keen: The Goodbye Galaxy Years",
        "description": "For Invasion of the Vorticons go here: https://youtu.be/iQQkInVetS4 For Aliens Ate My Babysitter go here: ...",
        "thumbnails": {
          "default": {
            "url": "https://i.ytimg.com/vi/7kBIu_4IBrM/default.jpg",
            "width": 120,
            "height": 90
          },
          "medium": {
            "url": "https://i.ytimg.com/vi/7kBIu_4IBrM/mqdefault.jpg",
            "width": 320,
            "height": 180
          },
          "high": {
            "url": "https://i.ytimg.com/vi/7kBIu_4IBrM/hqdefault.jpg",
            "width": 480,
            "height": 360
          }
        },
        "channelTitle": "Classic Ed",
        "liveBroadcastContent": "none",
        "publishTime": "2022-01-29T13:00:06Z"
      }
    },
    {
      "kind": "youtube#searchResult",
      "etag": "MJZiSx3IJhsYYqh-UiA1Yg0r_Sk",
      "id": {
        "kind": "youtube#video",
        "videoId": "gI9mPgS0pas"
      },
      "snippet": {
        "publishedAt": "2022-05-06T18:21:07Z",
        "channelId": "UCSdC_KL-it-m3Z-CK9VUUrg",
        "title": "How People Pushed 3D Movie Maker To Its Limits",
        "description": "3D Movie Maker is part of the Microsoft Kids series of software from the mid-90s. Despite being from 1995, this game is still being ...",
        "thumbnails": {
          "default": {
            "url": "https://i.ytimg.com/vi/gI9mPgS0pas/default.jpg",
            "width": 120,
            "height": 90
          },
          "medium": {
            "url": "https://i.ytimg.com/vi/gI9mPgS0pas/mqdefault.jpg",
            "width": 320,
            "height": 180
          },
          "high": {
            "url": "https://i.ytimg.com/vi/gI9mPgS0pas/hqdefault.jpg",
            "width": 480,
            "height": 360
          }
        },
        "channelTitle": "Classic Ed",
        "liveBroadcastContent": "none",
        "publishTime": "2022-05-06T18:21:07Z"
      }
    },
    {
      "kind": "youtube#searchResult",
      "etag": "ZaMkSlMSf-llfAb6BHg5BlQiETM",
      "id": {
        "kind": "youtube#video",
        "videoId": "p01V4u8AXK0"
      },
      "snippet": {
        "publishedAt": "2023-03-08T09:14:22Z",
        "channelId": "UCSdC_KL-it-m3Z-CK9VUUrg",
        "title": "In the Mind of Kimberly Kubus: The Game Developer Who Saw God",
        "description": "In this video, I delve into the fascinating world of game developer Kimberly Kubus and explore his work.",
        "thumbnails": {
          "default": {
            "url": "https://i.ytimg.com/vi/p01V4u8AXK0/default.jpg",
            "width": 120,
            "height": 90
          },
          "medium": {
            "url": "https://i.ytimg.com/vi/p01V4u8AXK0/mqdefault.jpg",
            "width": 320,
            "height": 180
          },
          "high": {
            "url": "https://i.ytimg.com/vi/p01V4u8AXK0/hqdefault.jpg",
            "width": 480,
            "height": 360
          }
        },
        "channelTitle": "Classic Ed",
        "liveBroadcastContent": "none",
        "publishTime": "2023-03-08T09:14:22Z"
      }
    },
    {
      "kind": "youtube#searchResult",
      "etag": "9QFTFIdyYVTLBEjf7W-i_6pWouk",
      "id": {
        "kind": "youtube#video",
        "videoId": "jgA0OUymoro"
      },
      "snippet": {
        "publishedAt": "2024-01-13T13:33:15Z",
        "channelId": "UCSdC_KL-it-m3Z-CK9VUUrg",
        "title": "What Happened to That&#39;s My Sonic?",
        "description": "That's My Sonic was a popular but also terrible and unfunny sprite comic from the early to late 00's. It catapulted it's creator, ...",
        "thumbnails": {
          "default": {
            "url": "https://i.ytimg.com/vi/jgA0OUymoro/default.jpg",
            "width": 120,
            "height": 90
          },
          "medium": {
            "url": "https://i.ytimg.com/vi/jgA0OUymoro/mqdefault.jpg",
            "width": 320,
            "height": 180
          },
          "high": {
            "url": "https://i.ytimg.com/vi/jgA0OUymoro/hqdefault.jpg",
            "width": 480,
            "height": 360
          }
        },
        "channelTitle": "Classic Ed",
        "liveBroadcastContent": "none",
        "publishTime": "2024-01-13T13:33:15Z"
      }
    },
    {
      "kind": "youtube#searchResult",
      "etag": "LdSySRmZwpSxk0gJZF1fOyXW0TA",
      "id": {
        "kind": "youtube#video",
        "videoId": "tLjEvqCvW1s"
      },
      "snippet": {
        "publishedAt": "2022-05-29T09:31:50Z",
        "channelId": "UCSdC_KL-it-m3Z-CK9VUUrg",
        "title": "What Lies at the Top of Manyland?",
        "description": "Climbing 457282 Blocks to Get to the Top of Manyland Manyland is a massively multiplayer online sandbox game where you can ...",
        "thumbnails": {
          "default": {
            "url": "https://i.ytimg.com/vi/tLjEvqCvW1s/default.jpg",
            "width": 120,
            "height": 90
          },
          "medium": {
            "url": "https://i.ytimg.com/vi/tLjEvqCvW1s/mqdefault.jpg",
            "width": 320,
            "height": 180
          },
          "high": {
            "url": "https://i.ytimg.com/vi/tLjEvqCvW1s/hqdefault.jpg",
            "width": 480,
            "height": 360
          }
        },
        "channelTitle": "Classic Ed",
        "liveBroadcastContent": "none",
        "publishTime": "2022-05-29T09:31:50Z"
      }
    }
  ]
}

In [66]:
from io import BytesIO
# This is an unordered list of dictionaries containing video information - enough to generate markdown pages and stuff with videos embedded
# The dictionary keys are:
# ['id']['videoId']: The YouTube video
# ['snippet']['title']: The title of the video
# ['snippet']['description']: The description of the video
# ['snippet']['thumbnails']['medium']['url']: The URL of the thumbnail - can download this and use it for the thumbnail
# ['snippet']['publishTime']: The time the video was published - can extract date and also use this for the feed and stuff

content = []
feed = {
}

for video in vid_json['items']:
    title = video['snippet']['title']
    description = video['snippet']['description']
    thumbnail_url = video['snippet']['thumbnails']['medium']['url']
    fullsize_thumbnail_url = video['snippet']['thumbnails']['high']['url']
    publish_time = video['snippet']['publishTime']
    _dt = dt.strptime(publish_time, '%Y-%m-%dT%H:%M:%SZ')
    ymd = _dt.strftime('%Y-%m-%d')
    timestamp = _dt.strftime('%Y-%m-%d %H:%M:%S')
    res = requests.get(thumbnail_url)
    img = Image.open(BytesIO(res.content))
    img.thumbnail((256, 256))
    img = img.convert('RGB')
    img.save(f"STAGING/images/{strip_punctuation(title).replace(' ', '_')}_thumbnail.jpg", 'JPEG')

    print(f"Downloaded thumbnail")

    res = requests.get(fullsize_thumbnail_url)
    img = Image.open(BytesIO(res.content))
    img.save(f"STAGING/images/{strip_punctuation(title).replace(' ', '_')}.png", 'PNG')

    print(f"Downloaded fullsize thumbnail")

    md = f"""---
date: {ymd}
title: {title}
description: {description}
author: Ed
tags: ['Video', 'YouTube', 'Video Game']
type: video
thumbnail: /assets/images/{strip_punctuation(title).replace(' ', '_')}_thumbnail.jpg
og_title: {title}
og_description: {description}
og_image: /assets/images/{strip_punctuation(title).replace(' ', '_')}.png
og_type: video
collection: Classic Ed
---
{video['id']['videoId']}
"""

    with open(f"STAGING/{strip_punctuation(title).replace(' ', '_')}.md", 'w') as f:
        f.write(md)

    print(f"Generated markdown for {title}")

    feed[timestamp] = {
        'location': f"videos/{strip_punctuation(title).replace(' ', '_')}.md",
    }

# Update content based on the feed


Downloaded thumbnail
Downloaded fullsize thumbnail
Generated markdown for Why don&#39;t we play our games anymore?
Downloaded thumbnail
Downloaded fullsize thumbnail
Generated markdown for This is Your Brain on Rollercoaster Tycoon
Downloaded thumbnail
Downloaded fullsize thumbnail
Generated markdown for Bringing Back the Dopefish Screensaver: A Wayback Machine Adventure
Downloaded thumbnail
Downloaded fullsize thumbnail
Generated markdown for Commander Keen&#39;s Last Adventure, Unveiling the Lipton Tea Connection
Downloaded thumbnail
Downloaded fullsize thumbnail
Generated markdown for An Unexpected Comment on Egyptian Theming Objects #shorts
Downloaded thumbnail
Downloaded fullsize thumbnail
Generated markdown for Why You Can&#39;t Play Fun School 6 Anymore
Downloaded thumbnail
Downloaded fullsize thumbnail
Generated markdown for Ratchet and Clank Nostalgia Video
Downloaded thumbnail
Downloaded fullsize thumbnail
Generated markdown for The Worst Box in Crash 2
Downloaded thumbnail
D

AttributeError: 'dict' object has no attribute 'split'

In [67]:
for timestamp in sorted(feed.keys(), reverse=True):
    content.append(feed[timestamp]['location'].split("/")[-1])
print(feed)
print(content)

{'2022-12-15 18:42:57': {'location': 'videos/Why_don39t_we_play_our_games_anymore.md'}, '2022-01-15 12:13:39': {'location': 'videos/This_is_Your_Brain_on_Rollercoaster_Tycoon.md'}, '2022-03-25 13:13:54': {'location': 'videos/Bringing_Back_the_Dopefish_Screensaver_A_Wayback_Machine_Adventure.md'}, '2023-07-02 11:24:15': {'location': 'videos/Commander_Keen39s_Last_Adventure_Unveiling_the_Lipton_Tea_Connection.md'}, '2022-01-22 13:04:12': {'location': 'videos/An_Unexpected_Comment_on_Egyptian_Theming_Objects_shorts.md'}, '2022-05-14 12:33:00': {'location': 'videos/Why_You_Can39t_Play_Fun_School_6_Anymore.md'}, '2022-11-03 11:37:50': {'location': 'videos/Ratchet_and_Clank_Nostalgia_Video.md'}, '2022-05-21 12:34:14': {'location': 'videos/The_Worst_Box_in_Crash_2.md'}, '2024-02-12 00:00:01': {'location': 'videos/The_People_Who_Make_Bad_Games.md'}, '2022-06-29 16:07:45': {'location': 'videos/Dunnet__The_Secret_Terminal_Game.md'}, '2023-12-14 19:58:28': {'location': 'videos/Reject_Social_Media

In [69]:
# Let's update the feed now
feed_ref = db.collection('feed').document('content-log')
old_feed = feed_ref.get().to_dict()
old_feed.update(feed)

In [70]:
old_feed

{'2023-12-11 10:10:20': {'location': 'comics/pp_comic14.md'},
 '2023-07-06 14:10:46': {'location': 'comics/pp_comic6.md'},
 '2004-08-16 00:00:07': {'location': 'comics/hewligg_urobokkle_8.md'},
 '2024-04-23 08:17:04': {'location': 'blogs/internet.md'},
 '2024-06-29 00:00:00': {'location': 'blogs/sql_recursive.md'},
 '2004-08-23 00:00:00': {'location': 'comics/hewligg_urobokkle_21_3.md'},
 '2004-08-16 00:00:01': {'location': 'comics/hewligg_urobokkle_2.md'},
 '2024-02-01 17:08:24': {'location': 'blogs/hip.md'},
 '2004-10-15 00:00:00': {'location': 'comics/hewligg_urobokkle_38.md'},
 '2023-07-18 08:52:17': {'location': 'blogs/onrss.md'},
 '2024-01-04 11:35:56': {'location': 'blogs/books.md'},
 '2004-10-21 00:00:00': {'location': 'comics/hewligg_urobokkle_44.md'},
 '2004-08-17 00:00:03': {'location': 'comics/hewligg_urobokkle_13.md'},
 '2023-07-21 18:02:45': {'location': 'comics/pp_comic11.md'},
 '2023-07-09 14:28:02': {'location': 'comics/pp_comic8.md'},
 '2004-10-06 00:00:00': {'locatio

In [71]:
# Rewrite old_feed to the database
feed_ref.set(old_feed)

update_time {
  seconds: 1721314864
  nanos: 602078000
}

In [72]:
content = content[::-1]
content

['How_Commander_Keen_Changed_the_Face_of_DOS_Gaming_Forever.md',
 'Doom_but_with_Commander_Keen_Music_shorts.md',
 'This_is_Your_Brain_on_Rollercoaster_Tycoon.md',
 'An_Unexpected_Comment_on_Egyptian_Theming_Objects_shorts.md',
 'A_Commander_Keen_5_Easter_Egg_andwait_what_shorts.md',
 'Commander_Keen_The_Goodbye_Galaxy_Years.md',
 'The_Slime_Volley_Ball_Experience_shorts.md',
 'How_Slime_Volleyball_Became_a_Cult_Classic.md',
 'The_Games_That_Nobody_Plays.md',
 'A_Closer_Look_at_the_Rare_666_Edition_of_Slime_Volleyball.md',
 'Bringing_Back_the_Dopefish_Screensaver_A_Wayback_Machine_Adventure.md',
 'How_People_Pushed_3D_Movie_Maker_To_Its_Limits.md',
 'Why_You_Can39t_Play_Fun_School_6_Anymore.md',
 'The_Worst_Box_in_Crash_2.md',
 'What_Lies_at_the_Top_of_Manyland.md',
 'StumbleUpon_Internet_Discovery_Through_the_0039s.md',
 'Dunnet__The_Secret_Terminal_Game.md',
 'Sonic39s_Blunder_Years.md',
 'This_Ratchet_and_Clank_Game_was_Lost_for_20_Years.md',
 'How_Fans_Resurrected_Jak_and_Daxter_in

In [73]:
# Now let's put it in collections
ref = db.collection('collections').document('classic_ed')
dic = {
    'content': content
}

ref.set(dic)

update_time {
  seconds: 1721314917
  nanos: 980872000
}

In [74]:
img = Image.open(os.path.join("STAGING", "images", "keyboard_cat.png"))
img.thumbnail((256, 256))
img.convert('RGB').save(os.path.join("STAGING", "images", "keyboard_cat_thumbnail.jpg"), 'JPEG')

In [76]:
# Updaate the feed and eds_blog collection
feed_ref = db.collection('feed').document('content-log')

# Find the keyboard_cat.md file and get the modified date for feed
time = os.path.getmtime(os.path.join("STAGING", "blogs", "keyboard_cat.md"))
_dt = dt.strftime(dt.fromtimestamp(time), '%Y-%m-%d %H:%M:%S')

In [77]:
feed = feed_ref.get().to_dict()
feed[_dt] = {
    'location': 'blogs/keyboard_cat.md'
}

In [78]:
feed

{'2004-08-16 00:00:07': {'location': 'comics/hewligg_urobokkle_8.md'},
 '2024-04-23 08:17:04': {'location': 'blogs/internet.md'},
 '2004-08-16 00:00:01': {'location': 'comics/hewligg_urobokkle_2.md'},
 '2023-07-02 11:24:15': {'location': 'videos/Commander_Keen39s_Last_Adventure_Unveiling_the_Lipton_Tea_Connection.md'},
 '2024-02-01 17:08:24': {'location': 'blogs/hip.md'},
 '2022-06-08 20:08:33': {'location': 'videos/StumbleUpon_Internet_Discovery_Through_the_0039s.md'},
 '2022-06-29 16:07:45': {'location': 'videos/Dunnet__The_Secret_Terminal_Game.md'},
 '2023-07-18 08:52:17': {'location': 'blogs/onrss.md'},
 '2023-03-08 09:14:22': {'location': 'videos/In_the_Mind_of_Kimberly_Kubus_The_Game_Developer_Who_Saw_God.md'},
 '2024-01-04 11:35:56': {'location': 'blogs/books.md'},
 '2004-08-17 00:00:03': {'location': 'comics/hewligg_urobokkle_13.md'},
 '2023-07-21 18:02:45': {'location': 'comics/pp_comic11.md'},
 '2023-07-09 14:28:02': {'location': 'comics/pp_comic8.md'},
 '2022-02-02 15:00:07'

In [79]:
# Write it back
feed_ref.set(feed)

update_time {
  seconds: 1721319485
  nanos: 53557000
}

In [80]:
# Then add this to the content list
content_ref = db.collection('collections').document('eds_blog')
content = content_ref.get().to_dict()['content']
content.append('keyboard_cat.md')
content

['stumbleupon.md',
 'bananas.md',
 'alcoholism.md',
 'oldwebsites.md',
 'onrss.md',
 'burnout.md',
 'dustydrawers.md',
 'stress.md',
 'prank.md',
 'depressed.md',
 'charlmes.md',
 'books.md',
 'hip.md',
 'horses.md',
 'internet.md',
 'discord.md',
 'keyboard_cat.md']

In [81]:
# Update
content_ref.set({
    'content': content
})

update_time {
  seconds: 1721319514
  nanos: 825036000
}